In [1]:
### Evaluate colocalization results
### Load the results saved in R2 script and check out some summaries

# Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"


Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.8-41. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel

corrplot 0.92 loaded

Loading required package: viridisLite


Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose



Attaching package: ‘MASS’


The following object is masked from ‘package:genefilter’:

    area



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘SeuratObject’


The following object is masked from 

# Parameters

In [12]:
cell_type_var ="CD4_T"
# c("CD4T","CD8T","monocyte","NK","B","DC")

In [13]:
dataset = 'oneK1K'

In [14]:
data_path = '../data/current/coeqtl_mapping/'

In [15]:
coeqtl_results_path = 'co_qtls_sceqtlgen/'

In [16]:
result_path =  '../data/current//results/'

# Data 

## GWAS Enrichment Data (Dan)

### Classified tests

In [9]:
gwas_enrichment_table = fread(paste0(data_path, coeqtl_results_path, '/meta_data/input/GWAS_enrichment/', 'gwas_enrichment_table.tsv.gz'))

In [17]:
head(gwas_enrichment_table,2)   # currently the table for CD4_T cells

feature_id,snp_id,p_value,beta,assessed_allele,chr_pos,egene,coegene,trans_qtl,co_qtl,gwas,ld_group
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>
CWF19L1_SC5D,10:100055816:G:A,0.19761807,-0.04347212,G,10:100055816,CWF19L1,SC5D,FALSE,FALSE,FALSE,3437
CWF19L1_FUT10,10:100055816:G:A,0.03557041,0.14179463,G,10:100055816,CWF19L1,FUT10,FALSE,FALSE,FALSE,3437


In [24]:
gwas_enrichment_table = gwas_enrichment_table[is.na(str_extract(gwas_enrichment_table$egene, 'HLA')),] # remove HLA genes

In [25]:
nrow(gwas_enrichment_table)

[1] 31344814

In [26]:
unique(gwas_enrichment_table$gwas)

[1] FALSE  TRUE

In [27]:
length(unique(gwas_enrichment_table$egene))  # 5906 egenes

[1] 5891

In [28]:
length(unique(gwas_enrichment_table$snp_id))  # amount unique variants

[1] 10148

In [30]:
length(unique(gwas_enrichment_table$ld_group))  # if variants are in the same ld group they are considered as one variant

[1] 7451

In [31]:
### Check overlap

In [33]:
unique_matches = unique(gwas_enrichment_table[,c('ld_group', 'co_qtl', 'gwas')])

In [36]:
head(unique_matches,2)

ld_group,co_qtl,gwas
<int>,<lgl>,<lgl>
3437,FALSE,FALSE
3438,FALSE,FALSE


In [38]:
unique_matches = unique_matches %>% group_by(ld_group) %>% summarise(co_qtl = max(co_qtl), gwas = max(gwas))    # check whether there is at least one gwas overlap

In [44]:
head(unique_matches,2)

ld_group,co_qtl,gwas
<int>,<int>,<int>
1,1,1
2,0,0


In [40]:
## Check enrichment table

In [41]:
enrichment_table = unique_matches  %>% group_by(co_qtl, gwas) %>% summarise(amount_variants = n())

`summarise()` has grouped output by 'co_qtl'. You can override using the
`.groups` argument.


In [45]:
enrichment_table = enrichment_table %>% dcast(co_qtl ~gwas, value.var = 'amount_variants')

In [46]:
enrichment_table

co_qtl,0,1
<int>,<int>,<int>
0,6170,706
1,486,89


In [54]:
input_matrix_test = as.matrix(enrichment_table)

In [56]:
input_matrix_test =input_matrix_test[,2:3]

In [57]:
input_matrix_test

0,1
6170,706
486,89


In [58]:
fisher_test_result = fisher.test(input_matrix_test)

In [61]:
fisher_test_result


	Fisher's Exact Test for Count Data

data:  input_matrix_test
p-value = 0.0002388
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 1.245324 2.038794
sample estimates:
odds ratio 
  1.600298 


In [62]:
### Also find enrichment but the numbers in the contingency table are different

### GWAS input

In [63]:
gwas_input = fread(paste0(data_path, coeqtl_results_path, '/meta_data/input/GWAS_enrichment/', 'gwas_filtered_updated_08032024.tsv.gz'))

In [64]:
head(gwas_input,2)

snp_id,chr,pos,trait,mapped_trait,mapped_trait_URI,allele,p_value,chr_pos,disease,immune_related
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>
rs12261919,10,100001413,Bioavailable testosterone levels,testosterone measurement,http://www.ebi.ac.uk/efo/EFO_0004908,rs12261919-T,7e-09,10_100001413,0,0
rs12261919,10,100001413,Free testosterone levels,testosterone measurement,http://www.ebi.ac.uk/efo/EFO_0004908,rs12261919-T,2e-08,10_100001413,0,0


In [65]:
nrow(gwas_input)

[1] 419729

In [111]:
gwas_input$pos_range_neg = gwas_input$pos -0
gwas_input$pos_range_pos = gwas_input$pos + 0

In [129]:
### check the mapped traits

In [132]:
sort(unique(gwas_input$mapped_trait[gwas_input$immune_related == 1]))

[1] "acute graft vs. host disease"                                                     
 [2] "acute myeloid leukemia"                                                           
 [3] "AIDS"                                                                             
 [4] "allergic disease"                                                                 
 [5] "allergic rhinitis"                                                                
 [6] "ankylosing spondylitis"                                                           
 [7] "anti-centromere-antibody-positive systemic scleroderma"                           
 [8] "anti-GAD65 autoimmune neurological syndromes"                                     
 [9] "anti-neutrophil antibody associated vasculitis"                                   
[10] "anti-topoisomerase-I-antibody-positive systemic scleroderma"                      
[11] "antiphospholipid syndrome"                                                        
[12] "appendicitis"                                                                     
[13] "AQP4-IgG-positive neuromyelitis optica"                                           
[14] "atopic eczema"                                                                    
[15] "atopy"                                                                            
[16] "autoimmune disease"                                                               
[17] "Autoimmune Hepatitis"                                                             
[18] "autoimmune thyroid disease"                                                       
[19] "B-cell acute lymphoblastic leukemia"                                              
[20] "B-cell acute lymphoblastic leukemia with t(1;19)(q23;p13.3); E2A-PBX1 (TCF3-PBX1)"
[21] "Behcet's syndrome"                                                                
[22] "bullous pemphigoid"                                                               
[23] "celiac disease"                                                                   
[24] "central nervous system non-hodgkin lymphoma"                                      
[25] "chronic lymphocytic leukemia"                                                     
[26] "chronic myelogenous leukemia"                                                     
[27] "Churg-Strauss syndrome"                                                           
[28] "clinically amyopathic dermatomyositis"                                            
[29] "collagenous colitis"                                                              
[30] "common variable immunodeficiency"                                                 
[31] "contact dermatitis"                                                               
[32] "Crohn's disease"                                                                  
[33] "cryoglobulinemia"                                                                 
[34] "cutaneous lupus erythematosus"                                                    
[35] "cutaneous mastocytosis"                                                           
[36] "dermatomyositis"                                                                  
[37] "diffuse large B-cell lymphoma"                                                    
[38] "diffuse scleroderma"                                                              
[39] "filarial elephantiasis"                                                           
[40] "Granulomatosis with Polyangiitis"                                                 
[41] "Graves disease"                                                                   
[42] "Hashimoto's thyroiditis"                                                          
[43] "Henoch-Schoenlein purpura"                                                        
[44] "immune system disease"                                                            
[45] "inclusion body myositis"                                                       

In [75]:
### test to reproduce the ovelrap

In [83]:
gwas_enrichment_table$chr = as.numeric(str_replace(gwas_enrichment_table$chr_pos, ':.*', ''))
gwas_enrichment_table$pos = as.numeric(str_replace(gwas_enrichment_table$chr_pos, '.*:', ''))

In [84]:
head(gwas_enrichment_table,2)

feature_id,snp_id,p_value,beta,assessed_allele,chr_pos,egene,coegene,trans_qtl,co_qtl,gwas,ld_group,chr,pos
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>,<dbl>,<dbl>
CWF19L1_SC5D,10:100055816:G:A,0.19761807,-0.04347212,G,10:100055816,CWF19L1,SC5D,FALSE,FALSE,FALSE,3437,10,100055816
CWF19L1_FUT10,10:100055816:G:A,0.03557041,0.14179463,G,10:100055816,CWF19L1,FUT10,FALSE,FALSE,FALSE,3437,10,100055816


In [124]:
### Overlap with the GWAS input to check which traits have been mapped

In [112]:
enrichment_test = left_join(gwas_enrichment_table, gwas_input, by = join_by(chr , pos <= pos_range_pos,
     pos >= pos_range_neg))

### Analyse

In [114]:
enrichment_test_gwas_overlap = enrichment_test[enrichment_test$gwas == TRUE,]
enrichment_test_no_gwas_overlap = enrichment_test[enrichment_test$gwas == FALSE,]

In [115]:
### Check the tests with gwas ovelrap

In [116]:
head(enrichment_test_gwas_overlap[is.na(enrichment_test_gwas_overlap$mapped_trait),])

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


feature_id,snp_id.x,p_value.x,beta,assessed_allele,chr_pos.x,egene,coegene,trans_qtl,co_qtl,⋯,trait,mapped_trait,mapped_trait_URI,allele,p_value.y,chr_pos.y,disease,immune_related,pos_range_neg,pos_range_pos
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>


In [117]:
nrow(enrichment_test_gwas_overlap[is.na(enrichment_test_gwas_overlap$mapped_trait),])

[1] 0

In [118]:
#head(enrichment_test_gwas_overlap,2)

In [119]:
head(enrichment_test_no_gwas_overlap[!is.na(enrichment_test_no_gwas_overlap$mapped_trait),])

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


feature_id,snp_id.x,p_value.x,beta,assessed_allele,chr_pos.x,egene,coegene,trans_qtl,co_qtl,⋯,trait,mapped_trait,mapped_trait_URI,allele,p_value.y,chr_pos.y,disease,immune_related,pos_range_neg,pos_range_pos
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>


In [120]:
nrow(enrichment_test_no_gwas_overlap[!is.na(enrichment_test_no_gwas_overlap$mapped_trait),])

[1] 0

In [123]:
### Mapping is done as exact mapping!

In [125]:
### Get the mapped traits for the found overlaps

In [159]:
head(enrichment_test ,2)

feature_id,snp_id.x,p_value.x,beta,assessed_allele,chr_pos.x,egene,coegene,trans_qtl,co_qtl,⋯,trait,mapped_trait,mapped_trait_URI,allele,p_value.y,chr_pos.y,disease,immune_related,pos_range_neg,pos_range_pos
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>
CWF19L1_SC5D,10:100055816:G:A,0.19761807,-0.04347212,G,10:100055816,CWF19L1,SC5D,FALSE,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CWF19L1_FUT10,10:100055816:G:A,0.03557041,0.14179463,G,10:100055816,CWF19L1,FUT10,FALSE,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [190]:
#head(enrichment_test[!is.na(enrichment_test$mapped_trait),])

In [191]:
#test = test[test$immune_related == 1,]

In [192]:
enrichment_test_summarized =  enrichment_test  %>%  group_by(feature_id, snp_id.x, egene, coegene, trans_qtl, co_qtl, gwas, ld_group) %>% summarise(
    gwas_overlap = max(!is.na(mapped_trait)),
    disease_related_overlap = max((!is.na(disease)) & (disease == 1)),
    immune_related_traits = paste0(mapped_trait[(!is.na(immune_related)) & (immune_related == 1) ], collapse = ','),
    immune_related_overlap = max((!is.na(immune_related)) & (immune_related == 1))
     )

`summarise()` has grouped output by 'feature_id', 'snp_id.x', 'egene',
'coegene', 'trans_qtl', 'co_qtl', 'gwas'. You can override using the `.groups`
argument.


In [193]:
## Check correct aggregation

In [194]:
nrow(enrichment_test_summarized)

[1] 31344814

In [195]:
nrow(gwas_enrichment_table)

[1] 31344814

In [196]:
### Check correct calculation

In [197]:
head(enrichment_test_summarized,2)

feature_id,snp_id.x,egene,coegene,trans_qtl,co_qtl,gwas,ld_group,gwas_overlap,disease_related_overlap,immune_related_traits,immune_related_overlap
<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>,<int>,<int>,<chr>,<int>
A1BG-AS1_AASS,19:58355068:G:A,A1BG-AS1,AASS,FALSE,FALSE,FALSE,860,0,0,,0
A1BG-AS1_AASS,7:122023523:T:TTA,AASS,A1BG-AS1,FALSE,FALSE,FALSE,4382,0,0,,0


In [198]:
head(enrichment_test_summarized[enrichment_test_summarized$immune_related_overlap == 1,])

feature_id,snp_id.x,egene,coegene,trans_qtl,co_qtl,gwas,ld_group,gwas_overlap,disease_related_overlap,immune_related_traits,immune_related_overlap
<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>,<int>,<int>,<chr>,<int>
A1BG-AS1_LAMB1,7:107880959:G:A,LAMB1,A1BG-AS1,FALSE,FALSE,TRUE,4367,1,1,inflammatory bowel disease,1
A1BG-AS1_PLEK,2:68371823:T:C,PLEK,A1BG-AS1,FALSE,FALSE,TRUE,6377,1,1,celiac disease,1
A1BG-AS1_SLC43A1,11:57435536:C:T,SLC43A1,A1BG-AS1,FALSE,FALSE,TRUE,2922,1,1,Crohn's disease,1
A1BG-AS1_TNFRSF14-AS1,1:2595307:A:G,TNFRSF14-AS1,A1BG-AS1,FALSE,FALSE,TRUE,6783,1,1,celiac disease,1
A1BG-AS1_YBX3,12:10444608:C:T,YBX3,A1BG-AS1,FALSE,FALSE,TRUE,2386,1,1,psoriasis,1
A1BG_AHI1,6:135581461:A:T,AHI1,A1BG,FALSE,FALSE,TRUE,4929,1,1,autoimmune disease,1


In [199]:
nrow(enrichment_test_summarized[(enrichment_test_summarized$gwas == TRUE) & (enrichment_test_summarized$gwas_overlap == 0),])
nrow(enrichment_test_summarized[(enrichment_test_summarized$gwas == FALSE) & (enrichment_test_summarized$gwas_overlap == 1),])

[1] 0

[1] 0

In [200]:
nrow(enrichment_test_summarized[(enrichment_test_summarized$gwas == TRUE),])
nrow(enrichment_test_summarized[(enrichment_test_summarized$gwas_overlap == 1),])

[1] 2830548

[1] 2830548

In [205]:
## Check enrichment for gwas related overlap (to double-check)

In [202]:
unique_matches = unique(enrichment_test_summarized[,c('ld_group', 'co_qtl', 'gwas_overlap')])

unique_matches = unique_matches %>% group_by(ld_group) %>% summarise(co_qtl = max(co_qtl), gwas_overlap = max(gwas_overlap))    # check whether there is at least one gwas overlap

## Check enrichment table

enrichment_table = unique_matches  %>% group_by(co_qtl, gwas_overlap) %>% summarise(amount_variants = n())

enrichment_table = enrichment_table %>% dcast(co_qtl ~gwas_overlap, value.var = 'amount_variants')

`summarise()` has grouped output by 'co_qtl'. You can override using the
`.groups` argument.


In [203]:
enrichment_table

co_qtl,0,1
<int>,<int>,<int>
0,6170,706
1,486,89


In [204]:
## Test:
input_matrix_test = as.matrix(enrichment_table)
input_matrix_test =input_matrix_test[,2:3]
fisher_test_result = fisher.test(input_matrix_test)

fisher_test_result


	Fisher's Exact Test for Count Data

data:  input_matrix_test
p-value = 0.0002388
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 1.245324 2.038794
sample estimates:
odds ratio 
  1.600298 


In [205]:
## Check enrichment for disease related overlap (to double-check)

In [206]:
unique_matches = unique(enrichment_test_summarized[,c('ld_group', 'co_qtl', 'disease_related_overlap')])

unique_matches = unique_matches %>% group_by(ld_group) %>% summarise(co_qtl = max(co_qtl), disease_related_overlap  = max(disease_related_overlap))    # check whether there is at least one gwas overlap

## Check enrichment table

enrichment_table = unique_matches  %>% group_by(co_qtl, disease_related_overlap) %>% summarise(amount_variants = n())

enrichment_table = enrichment_table %>% dcast(co_qtl ~disease_related_overlap, value.var = 'amount_variants')

`summarise()` has grouped output by 'co_qtl'. You can override using the
`.groups` argument.


In [207]:
enrichment_table

co_qtl,0,1
<int>,<int>,<int>
0,6747,129
1,555,20


In [208]:
## Test:
input_matrix_test = as.matrix(enrichment_table)
input_matrix_test =input_matrix_test[,2:3]
fisher_test_result = fisher.test(input_matrix_test)

fisher_test_result


	Fisher's Exact Test for Count Data

data:  input_matrix_test
p-value = 0.01266
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 1.105177 3.061557
sample estimates:
odds ratio 
  1.884646 


In [205]:
## Check enrichment for disease related overlap (to double-check)

In [209]:
unique_matches = unique(enrichment_test_summarized[,c('ld_group', 'co_qtl', 'immune_related_overlap')])

unique_matches = unique_matches %>% group_by(ld_group) %>% summarise(co_qtl = max(co_qtl), immune_related_overlap  = max(immune_related_overlap))    # check whether there is at least one gwas overlap

## Check enrichment table

enrichment_table = unique_matches  %>% group_by(co_qtl, immune_related_overlap) %>% summarise(amount_variants = n())

enrichment_table = enrichment_table %>% dcast(co_qtl ~ immune_related_overlap, value.var = 'amount_variants')

`summarise()` has grouped output by 'co_qtl'. You can override using the
`.groups` argument.


In [210]:
enrichment_table

co_qtl,0,1
<int>,<int>,<int>
0,6846,30
1,564,11


In [211]:
## Test:
input_matrix_test = as.matrix(enrichment_table)
input_matrix_test =input_matrix_test[,2:3]
fisher_test_result = fisher.test(input_matrix_test)

fisher_test_result


	Fisher's Exact Test for Count Data

data:  input_matrix_test
p-value = 0.0001955
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 2.000621 9.192330
sample estimates:
odds ratio 
  4.449193 


In [212]:
### Check out the immune related overlaps

In [219]:
unique(enrichment_test_summarized[enrichment_test_summarized$immune_related_overlap == 1,c( 'egene',  'ld_group', 'co_qtl', 'immune_related_overlap', 'immune_related_traits')])

egene,ld_group,co_qtl,immune_related_overlap,immune_related_traits
<chr>,<int>,<lgl>,<int>,<chr>
LAMB1,4367,FALSE,1,inflammatory bowel disease
PLEK,6377,FALSE,1,celiac disease
SLC43A1,2922,FALSE,1,Crohn's disease
TNFRSF14-AS1,6783,FALSE,1,celiac disease
YBX3,2386,FALSE,1,psoriasis
AHI1,4929,FALSE,1,autoimmune disease
C2orf74,6353,FALSE,1,"Crohn's disease,Crohn's disease"
CCR6,4998,FALSE,1,"rheumatoid arthritis,rheumatoid arthritis"
CISD1,3313,FALSE,1,Crohn's disease


In [220]:
#Get egenes with immune related overlap acc to Dans data

In [222]:
egene_immune_overlap = unique(enrichment_test_summarized[(enrichment_test_summarized$immune_related_overlap == 1) & (enrichment_test_summarized$co_qtl == TRUE)
                                  ,c( 'egene',  'ld_group', 'co_qtl', 'immune_related_overlap', 'immune_related_traits')])

In [363]:
egene_immune_overlap

egene,ld_group,co_qtl,immune_related_overlap,immune_related_traits
<chr>,<int>,<lgl>,<int>,<chr>
DOK6,988,TRUE,1,systemic lupus erythematosus
RNASET2,4995,TRUE,1,inflammatory bowel disease
ERAP2,5189,TRUE,1,"psoriasis,ankylosing spondylitis"
IQCB1,5993,TRUE,1,multiple sclerosis
CTSW,2979,TRUE,1,Crohn's disease
CD226,988,TRUE,1,"autoimmune thyroid disease,type 1 diabetes mellitus,type 1 diabetes mellitus"
PHTF1,7157,TRUE,1,systemic lupus erythematosus
RPS26,2529,TRUE,1,"allergic disease,Vitiligo"
PLEK,6377,TRUE,1,celiac disease


In [ ]:
## no coloc: DOK6; CTSW (Crohn's disease!); CD226 (type1 diabetes); PHTF1 , PLEK, LST1

## diff coloc: ERAP2; RPS26 ; NACA


## matches: RNASET, IQCB1

## coloc no overlap, e.g.: AFF3; AUH; ENO1, ....

In [355]:
### Compare to found colocalization

In [357]:
coloc_result = coeqtl_summary_filtered_H4[is.na(str_extract(coeqtl_summary_filtered_H4$egene, 'HLA')),]

In [360]:
colocalization_signal = unique(coloc_result[,c('egene', 'coloc_cat', 'trait')])

In [361]:
overlap_check = merge(colocalization_signal, egene_immune_overlap, by = 'egene', all.x = TRUE)

In [364]:
overlap_check %>% arrange(egene)

egene,coloc_cat,trait,ld_group,co_qtl,immune_related_overlap,immune_related_traits
<chr>,<chr>,<chr>,<int>,<lgl>,<int>,<chr>
AFF3,intermediate_coloc_signal,Rheumatoid Arthritis,NA,NA,NA,NA
AFF3,low_coloc_signal,Rheumatoid Arthritis,NA,NA,NA,NA
ASH1L,low_coloc_signal,Crohn's Disease,NA,NA,NA,NA
AUH,intermediate_coloc_signal,Inflammatory Bowel Disease,NA,NA,NA,NA
DIP2A,low_coloc_signal,White blood cell count,NA,NA,NA,NA
DIP2A,intermediate_coloc_signal,White blood cell count,NA,NA,NA,NA
DOCK8,high_coloc_signal,White blood cell count,NA,NA,NA,NA
ENO1,intermediate_coloc_signal,White blood cell count,NA,NA,NA,NA
ENO1,high_coloc_signal,White blood cell count,NA,NA,NA,NA


In [358]:
head(coloc_result,2)

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min,coloc_cat
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
AFF3_LMNA,Rheumatoid Arthritis,PP.H4.abf,0.7636360,,1080,1080,0.7636360,0.7636360,1080,2:100190052:T:C,100190052,AFF3,LMNA,100215693,100215693,100190052,-25641,intermediate_coloc_signal
AFF3_SH2D3A,Rheumatoid Arthritis,PP.H4.abf,0.7129591,,1147,1147,0.7129591,0.7129591,1147,2:100208905:C:T,100208905,AFF3,SH2D3A,100215693,100215693,100208905,-6788,low_coloc_signal


## Load list of co-eQTLs

In [236]:
### To this meta-dataset all the information should be joined

In [237]:
sign_coeqtls = read.csv(paste0(result_path, 'G1/', dataset, '_', cell_type_var, "_significant_coeqtls.csv"))

In [238]:
head(sign_coeqtls,2)

,X,feature_id,snp_id,beta,p_value,q_val_filter,egene,co_egene
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,0,AAK1_EIF5A,17:7304878:C:T,-0.2303832,1.020164e-13,1.095937e-09,EIF5A,AAK1
2,1,RERE_SMAD4,1:8400837:CAAA:C,-0.1382328,1.246527e-05,3.056836e-02,RERE,SMAD4


In [239]:
sign_coeqtls[sign_coeqtls$feature_id == 'RPS26_SMDT1',]

,X,feature_id,snp_id,beta,p_value,q_val_filter,egene,co_egene
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
475,474,RPS26_SMDT1,22:42080750:A:C,0.2615365,2.285982e-17,3.251999e-13,SMDT1,RPS26
476,475,RPS26_SMDT1,22:42102200:C:T,0.2614214,2.363205e-17,3.358796e-13,SMDT1,RPS26
3310,3309,RPS26_SMDT1,22:42092902:G:A,0.2617154,2.170911e-17,3.099592e-13,SMDT1,RPS26
5065,5064,RPS26_SMDT1,22:42108675:A:G,0.2606858,2.921028e-17,4.129080e-13,SMDT1,RPS26
6290,6289,RPS26_SMDT1,12:56007301:G:A,0.4925056,2.975672e-63,1.874189e-58,RPS26,SMDT1
6497,6496,RPS26_SMDT1,22:42067810:C:T,-0.2638423,1.171033e-17,1.692089e-13,SMDT1,RPS26


In [240]:
length(unique(sign_coeqtls$feature_id))

[1] 4157

In [241]:
nrow(sign_coeqtls[sign_coeqtls$egene == 'RPS26',])

[1] 572

In [242]:
#sign_coeqtls[sign_coeqtls$feature_id == 'RPS26_SMDT1',]

In [243]:
### Calculate amount of co-egenes per egene

In [244]:
amount_co_egenes = unique(sign_coeqtls[,c('egene', 'co_egene')]) %>% group_by(egene) %>% summarise(amount_co_egenes = n())

In [245]:
head(amount_co_egenes %>% arrange(-amount_co_egenes),10)

egene,amount_co_egenes
<chr>,<int>
RPS26,572
EIF5A,482
SMDT1,136
SH3YL1,122
SNHG8,94
KCNQ5,86
RNASET2,70
ANK3,64
MYOM2,63


In [246]:
#head(amount_co_egenes)

In [247]:
### Calculate amount of egenes per co-egene

In [248]:
amount_egenes = unique(sign_coeqtls[,c('egene', 'co_egene')]) %>% group_by(co_egene) %>% summarise(amount_egenes = n())

In [249]:
head(amount_egenes %>% arrange(-amount_egenes),6)

co_egene,amount_egenes
<chr>,<int>
EEF1G,51
BACH2,48
ATXN1,39
FHIT,39
ANXA1,37
IL32,34


In [250]:
### Get unique feature_id, egene, snp position assignments (if multiple SNPs have been tested the min and max SNP)

In [251]:
sign_coeqtls$pos = str_extract(sign_coeqtls$snp_id, ':\\d.*\\d:')
sign_coeqtls$pos = str_replace_all(sign_coeqtls$pos, ':', '')
#sign_coeqtls$pos = str_replace(sign_coeqtls$pos, '.*:', '')

In [252]:
head(sign_coeqtls,2)

,X,feature_id,snp_id,beta,p_value,q_val_filter,egene,co_egene,pos
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,0,AAK1_EIF5A,17:7304878:C:T,-0.2303832,1.020164e-13,1.095937e-09,EIF5A,AAK1,7304878
2,1,RERE_SMAD4,1:8400837:CAAA:C,-0.1382328,1.246527e-05,3.056836e-02,RERE,SMAD4,8400837


In [253]:
feature_id_egene = sign_coeqtls %>% group_by(feature_id, egene, co_egene) %>% summarise(min_pos = min(pos), max_pos = max(pos))

`summarise()` has grouped output by 'feature_id', 'egene'. You can override
using the `.groups` argument.


In [254]:
head(feature_id_egene)

feature_id,egene,co_egene,min_pos,max_pos
<chr>,<chr>,<chr>,<chr>,<chr>
AAK1_EIF5A,EIF5A,AAK1,7304878,7304878
ABI1_YWHAQ,YWHAQ,ABI1,9704049,9704049
ABLIM1_AFAP1,AFAP1,ABLIM1,7930977,7930977
ABLIM1_CARMIL1,CARMIL1,ABLIM1,25398198,25398777
ABLIM1_CENPK,CENPK,ABLIM1,65541278,65614578
ABLIM1_KCNQ5,KCNQ5,ABLIM1,72601379,72601379


## Summaries

### EQTL coloclaization summaries 

In [28]:
load_path = paste0('/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/results/G1/',dataset, '_', cell_type_var,  'EQTL_summary_update.csv')
eqtl_summary = fread(load_path)
print(load_path)
print(file.info(load_path )$ctime)

[1] "/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/results/G1/oneK1K_BEQTL_summary_update.csv"
[1] "2024-06-12 10:18:16 CEST"


In [29]:
head(eqtl_summary,2)

parameter,value,trait,identifier
<chr>,<dbl>,<chr>,<chr>
nsnps,5.040000e+02,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,OPTN
PP.H0.abf,1.051531e-45,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,OPTN


In [30]:
#eqtl_summary[eqtl_summary$identifier == 'TUFM',]

In [31]:
### Exk: check inclusion of genes Maryna

In [32]:
egenes_maryna = read.csv("/ictstr01/groups/epigenereg01/workspace/projects/grn_dev_groningen/multiome_analysis/overlapped_eGenes_in_regulon_export.tsv", sep = '\t')

In [33]:
eqtl_summary_maryna = eqtl_summary[eqtl_summary$identifier %in% egenes_maryna$x,]

In [34]:
#eqtl_summary_maryna[eqtl_summary_maryna$parameter == 'PP.H4.abf',] %>% arrange(-value)

In [35]:
head(eqtl_summary_maryna)

parameter,value,trait,identifier
<chr>,<dbl>,<chr>,<chr>
nsnps,5.700000e+02,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,PRKCB
PP.H0.abf,2.109177e-01,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,PRKCB
PP.H1.abf,6.864653e-04,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,PRKCB
PP.H2.abf,7.856280e-01,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,PRKCB
PP.H3.abf,2.554821e-03,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,PRKCB
PP.H4.abf,2.130209e-04,UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema,PRKCB


In [36]:
write.csv(eqtl_summary_maryna, 'Coloc_Summary_eGenes.csv')

In [37]:
#eqtl_summary_maryna[eqtl_summary_maryna$parameter == 'PP.H4.abf',] %>% arrange(-value)

In [38]:
#head(egenes_maryna,2)

In [39]:
length(unique(egenes_maryna$x))

[1] 45

In [40]:
unique(egenes_maryna$x)[unique(egenes_maryna$x) %in% eqtl_summary$identifier]

[1] "PRKCB"      "RPS5"       "GYPC"       "HLA-DPA1"   "CSGALNACT1"

In [41]:
unique(eqtl_summary$trait)

[1] "UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema"
 [2] "UKB_6152_8_diagnosed_by_doctor_Asthma"                              
 [3] "UKB_2395_3_Hair_or_balding_pattern_Pattern_3"                       
 [4] "UKB_2395_2_Hair_or_balding_pattern_Pattern_2"                       
 [5] "UKB_21001_Body_mass_index_BMI"                                      
 [6] "UKB_20002_1313_self_reported_ankylosing_spondylitis"                
 [7] "UKB_20002_1261_self_reported_multiple_sclerosis"                    
 [8] "UKB_20002_1226_self_reported_hypothyroidism_or_myxoedema"           
 [9] "UKB_20002_1225_self_reported_hyperthyroidism_or_thyrotoxicosis"     
[10] "UKB_20002_1223_self_reported_type_2_diabetes"                       
[11] "UKB_1180_Morning_or_evening_person_chronotype"                      
[12] "UKB_1160_Sleep_duration"                                            
[13] "TAGC_Asthma_EUR"                                                    
[14] "RA_OKADA_TRANS_ETHNIC"                                              
[15] "IMMUNOBASE_Systemic_lupus_erythematosus_hg19"                       
[16] "IMMUNOBASE_Multiple_sclerosis_hg19"                                 
[17] "IMMUNOBASE_Celiac_disease_hg19"                                     
[18] "ICBP_SystolicPressure"                                              
[19] "IBD.EUR.Inflammatory_Bowel_Disease"                                 
[20] "IBD.EUR.Crohns_Disease"                                             
[21] "GIANT_2017_BMI_Active_EUR"                                          
[22] "GABRIEL_Asthma"                                                     
[23] "chiou_type_1_diabetes_study"                                        
[24] "BCAC_ER_positive_BreastCancer_EUR"                                  
[25] "Astle_et_al_2016_White_blood_cell_count"                            
[26] "Astle_et_al_2016_Sum_eosinophil_basophil_counts"                    
[27] "Astle_et_al_2016_Reticulocyte_count"                                
[28] "Astle_et_al_2016_Lymphocyte_counts"                                 
[29] "Astle_et_al_2016_High_light_scatter_reticulocyte_count"             
[30] "Astle_et_al_2016_Eosinophil_counts"

In [42]:
eqtl_summary = unique(eqtl_summary)

In [43]:
duplicates = eqtl_summary %>% group_by(parameter, trait, identifier) %>% dplyr::count() %>% filter(n>= 2)

In [44]:
unique(duplicates$trait)

character(0)

### Co-EQTL colocalization summaries

In [255]:
coeqtl_summary = data.frame()

#for(i in all_files){
    load_path = paste0(result_path, '/G1/', dataset, '_', cell_type_var, 'COEQTL_summary_update_overlap',  '.csv')   # 'COEQTL_summary_update_overlap' ; old e.g. B: COEQTL_summary_update1_968; CD4_T: COEQTL_summary_update1_4157 
    data = fread(load_path)
    coeqtl_summary = rbind(coeqtl_summary, data)
    print(file.info(load_path )$ctime)
#    }

[1] "2024-06-11 19:32:26 CEST"


In [256]:
coeqtl_summary$X = NULL

Warning message in set(x, j = name, value = value):
“Column 'X' does not exist to remove”


In [257]:
head(coeqtl_summary,2)

parameter,value,trait,identifier
<chr>,<dbl>,<chr>,<chr>
nsnps,9.840000e+02,White blood cell count,CELF2_EIF5A
PP.H0.abf,4.485855e-07,White blood cell count,CELF2_EIF5A


In [258]:
#coeqtl_summary[coeqtl_summary$identifier == 'RPS26_SMDT1',]

In [259]:
### Check for duplicates

In [260]:
coeqtl_summary = unique(coeqtl_summary)

In [261]:
duplicates = coeqtl_summary %>% group_by(parameter, trait, identifier) %>% dplyr::count() %>% filter(n>= 2)

In [262]:
unique(duplicates$trait)

character(0)

In [263]:
#unique(duplicates$parameter)

In [264]:
head(duplicates,2)

parameter,trait,identifier,n
<chr>,<chr>,<chr>,<int>


In [265]:
### filter co-eqtl summary on only feature_ids from sign after decision tree filtering

In [266]:
length(unique(coeqtl_summary$identifier))

[1] 4157

In [267]:
coeqtl_summary = coeqtl_summary[coeqtl_summary$identifier %in% sign_coeqtls$feature_id,]

In [268]:
length(unique(coeqtl_summary$identifier))

[1] 4157

In [269]:
head(coeqtl_summary,2)

parameter,value,trait,identifier
<chr>,<dbl>,<chr>,<chr>
nsnps,9.840000e+02,White blood cell count,CELF2_EIF5A
PP.H0.abf,4.485855e-07,White blood cell count,CELF2_EIF5A


In [270]:
#coeqtl_summary[coeqtl_summary$identifier == 'RPL10A_TUFM',]

## Detailled summaries 

In [271]:
### For co-eqtls (included information about most significant snps)

In [272]:
load_path = paste0(result_path, "/G1/",  dataset, '_', cell_type_var,  "COEQTL_detail_update_overlap",  ".csv")  # COEQTL_detail_update_overlap; old B-cells: COEQTL_detail_update1_968; old CD4_T: COEQTL_detail_update1_4157
detailled_summary = fread(load_path)
print(file.info(load_path )$ctime)

[1] "2024-06-11 19:32:41 CEST"


In [273]:
head(detailled_summary,2)

snp,position,V.df1,z.df1,r.df1,lABF.df1,V.df2,z.df2,r.df2,lABF.df2,internal.sum.lABF,SNP.PP.H4,trait,identifier
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
10:31911489:A:AT,31911489,1.277956e-05,2.185745,0.9996806,-1.636575,0.002436242,1.544070,0.9023012,-0.0873212,-1.723896,8.587379e-12,White blood cell count,ITGB1_RBMS1
10:31913141:C:T,31913141,1.276953e-05,2.124021,0.9996809,-1.769934,0.002441186,1.532223,0.9021223,-0.1030580,-1.872992,7.397910e-12,White blood cell count,ITGB1_RBMS1


In [274]:


min(detailled_summary$SNP.PP.H4)
max(detailled_summary$SNP.PP.H4)

[1] 0

[1] 1

In [275]:
### by identifier and trait get SNP with highest colocalization probability

In [276]:
max_prob_per_ident = detailled_summary %>% group_by(identifier, trait) %>% filter(SNP.PP.H4 == max(SNP.PP.H4))

In [277]:
## make unique
max_prob_per_ident = max_prob_per_ident %>% group_by(trait, identifier) %>% summarise(snp = max(snp), position = max(position))

`summarise()` has grouped output by 'trait'. You can override using the
`.groups` argument.


In [278]:
head(max_prob_per_ident)

trait,identifier,snp,position
<chr>,<chr>,<chr>,<int>
Asthma,ANXA1_HLA-DRB1,6:32658472:T:C,32658472
Asthma,CNTRL_WDR36,5:111069301:A:C,111069301
Asthma,EEF1G_RPS26,12:56007301:G:A,56007301
Asthma,FAU_HLA-DQB1,6:32658472:T:C,32658472
Asthma,FAU_RPS26,12:56007301:G:A,56007301
Asthma,GNLY_RUNX2,6:45584874:G:A,45584874


In [279]:
nrow(max_prob_per_ident)

[1] 1815

In [280]:
nrow(unique(max_prob_per_ident[,c('identifier', 'trait')]))

[1] 1815

# Evaluate summaries

## For EQTLs

In [281]:
head(eqtl_summary,2)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'eqtl_summary' not found


In [282]:
## Extract amount of SNPS

In [73]:
n_snps = eqtl_summary[eqtl_summary$parameter == 'nsnps',]
n_snps$overlapping_snps = n_snps$value
n_snps$parameter = NULL
n_snps$value = NULL

In [74]:
#eqtl_summary = eqtl_summary[eqtl_summary$parameter != 'nsnps',]

In [75]:
## Extract cell-type, gene etc

In [76]:
eqtl_summary$gene = str_replace(eqtl_summary$identifier, '.*_', '')

In [77]:
## Add number of SNPS

In [78]:
eqtl_summary = merge(eqtl_summary, n_snps, by.x = c('trait', 'identifier'), by.y = c('trait', 'identifier'))

In [79]:
unique(eqtl_summary$trait)

[1] "Astle_et_al_2016_Eosinophil_counts"                                 
 [2] "Astle_et_al_2016_High_light_scatter_reticulocyte_count"             
 [3] "Astle_et_al_2016_Lymphocyte_counts"                                 
 [4] "Astle_et_al_2016_Reticulocyte_count"                                
 [5] "Astle_et_al_2016_Sum_eosinophil_basophil_counts"                    
 [6] "Astle_et_al_2016_White_blood_cell_count"                            
 [7] "BCAC_ER_positive_BreastCancer_EUR"                                  
 [8] "GABRIEL_Asthma"                                                     
 [9] "GIANT_2017_BMI_Active_EUR"                                          
[10] "IBD.EUR.Crohns_Disease"                                             
[11] "IBD.EUR.Inflammatory_Bowel_Disease"                                 
[12] "ICBP_SystolicPressure"                                              
[13] "IMMUNOBASE_Celiac_disease_hg19"                                     
[14] "IMMUNOBASE_Multiple_sclerosis_hg19"                                 
[15] "IMMUNOBASE_Systemic_lupus_erythematosus_hg19"                       
[16] "RA_OKADA_TRANS_ETHNIC"                                              
[17] "TAGC_Asthma_EUR"                                                    
[18] "UKB_1160_Sleep_duration"                                            
[19] "UKB_1180_Morning_or_evening_person_chronotype"                      
[20] "UKB_20002_1223_self_reported_type_2_diabetes"                       
[21] "UKB_20002_1225_self_reported_hyperthyroidism_or_thyrotoxicosis"     
[22] "UKB_20002_1226_self_reported_hypothyroidism_or_myxoedema"           
[23] "UKB_20002_1261_self_reported_multiple_sclerosis"                    
[24] "UKB_20002_1313_self_reported_ankylosing_spondylitis"                
[25] "UKB_21001_Body_mass_index_BMI"                                      
[26] "UKB_2395_2_Hair_or_balding_pattern_Pattern_2"                       
[27] "UKB_2395_3_Hair_or_balding_pattern_Pattern_3"                       
[28] "UKB_6152_8_diagnosed_by_doctor_Asthma"                              
[29] "UKB_6152_9_diagnosed_by_doctor_Hayfever_allergic_rhinitis_or_eczema"
[30] "chiou_type_1_diabetes_study"

In [80]:
head(eqtl_summary,2)

trait,identifier,parameter,value,gene,overlapping_snps
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
Astle_et_al_2016_Eosinophil_counts,ABI1,nsnps,2.710000e+02,ABI1,271
Astle_et_al_2016_Eosinophil_counts,ABI1,PP.H0.abf,3.532438e-16,ABI1,271


In [81]:
#eqtl_summary[eqtl_summary$identifier == 'SESN1',]

In [82]:
### Maximum probability per identifier and trait

In [83]:
max_prob = eqtl_summary[eqtl_summary$parameter != 'nsnps',] %>% group_by(trait, identifier, gene) %>% summarise(value = max(value))

`summarise()` has grouped output by 'trait', 'identifier'. You can override
using the `.groups` argument.


In [84]:
### Filter on the hypothesis with max probability

In [85]:
eqtl_summary_filtered = merge(eqtl_summary, max_prob)

In [86]:
eqtl_summary_filtered = eqtl_summary_filtered[eqtl_summary_filtered$value.x == eqtl_summary_filtered$value.y,]

In [87]:
eqtl_summary_filtered$value = eqtl_summary_filtered$value.x

In [88]:
head(eqtl_summary_filtered,2)

trait,identifier,parameter,value.x,gene.x,overlapping_snps,gene.y,value.y,value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
Astle_et_al_2016_Eosinophil_counts,ABI1,PP.H3.abf,0.9539714,ABI1,271,ABI1,0.9539714,0.9539714
Astle_et_al_2016_Eosinophil_counts,ABRACL,PP.H2.abf,0.7040967,ABRACL,332,ABRACL,0.7040967,0.7040967


In [89]:
## Filter on cases where there is the highest indication for H4

In [90]:
eqtl_summary_filtered_H4 = eqtl_summary_filtered[eqtl_summary_filtered$parameter == 'PP.H4.abf',]

In [91]:
nrow(eqtl_summary_filtered_H4)

[1] 94

In [92]:
head(eqtl_summary_filtered_H4,2)

trait,identifier,parameter,value.x,gene.x,overlapping_snps,gene.y,value.y,value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
Astle_et_al_2016_Eosinophil_counts,COX4I1,PP.H4.abf,0.6421644,COX4I1,354,COX4I1,0.6421644,0.6421644
Astle_et_al_2016_Eosinophil_counts,MDM4,PP.H4.abf,0.7205195,MDM4,388,MDM4,0.7205195,0.7205195


In [93]:
length(unique(eqtl_summary_filtered_H4$identifier))   # amount of testes eGenes (eQTLs with colocalization signal)

[1] 49

In [94]:
eqtl_summary_filtered_H4 = eqtl_summary_filtered_H4[eqtl_summary_filtered_H4$value > 0.5,]  # keep only those cases with probability > 0.5

In [95]:
length(unique(eqtl_summary_filtered_H4$identifier))  # amount of testes eGenes (eQTLs with colocalization signal)

[1] 45

In [96]:
unique(eqtl_summary_filtered_H4$identifier)

[1] "COX4I1"   "MDM4"     "PTPN1"    "RASGRP1"  "RPS26"    "TMEM258" 
 [7] "APRT"     "PRDX5"    "RPL23A"   "RPS10"    "TYMP"     "APOLD1"  
[13] "ITM2B"    "KDELR2"   "UPF3A"    "ZBTB38"   "LYST"     "OPTN"    
[19] "PPIA"     "TESK2"    "HLA-DQB1" "TSPO"     "CSK"      "CD37"    
[25] "CLEC2D"   "EAF2"     "HLA-DRB5" "ZFP36L1"  "BANK1"    "ST8SIA4" 
[31] "AFF3"     "FCRL3"    "RPL3"     "AP1S3"    "HLA-DQA1" "HLA-DQA2"
[37] "UBE2G1"   "ZCCHC7"   "RAC2"     "FUT8"     "YWHAB"    "ZFAS1"   
[43] "CD86"     "CTSH"     "SIK3"

In [97]:
### Categorize colocalization signal for eGene

In [98]:
head(eqtl_summary_filtered_H4,2)

trait,identifier,parameter,value.x,gene.x,overlapping_snps,gene.y,value.y,value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
Astle_et_al_2016_Eosinophil_counts,COX4I1,PP.H4.abf,0.6421644,COX4I1,354,COX4I1,0.6421644,0.6421644
Astle_et_al_2016_Eosinophil_counts,MDM4,PP.H4.abf,0.7205195,MDM4,388,MDM4,0.7205195,0.7205195


In [99]:
### categorize coloc signal
eqtl_summary_filtered_H4$coloc_cat = 'low_coloc_signal'  # at least greater than 0.5 (already filtered)
eqtl_summary_filtered_H4$coloc_cat[eqtl_summary_filtered_H4$value > 0.75] = 'intermediate_coloc_signal'
eqtl_summary_filtered_H4$coloc_cat[eqtl_summary_filtered_H4$value > 0.9] = 'high_coloc_signal'

In [100]:
### summarise the tratis per coloc signal

In [101]:
coloc_signal_summarized_eqtl = eqtl_summary_filtered_H4%>% group_by(identifier, coloc_cat) %>% summarise(traits = paste0(trait, collapse = ';'))

`summarise()` has grouped output by 'identifier'. You can override using the
`.groups` argument.


In [102]:
coloc_signal_summarized_eqtl  = coloc_signal_summarized_eqtl  %>% dcast(identifier  ~ coloc_cat, value.var = 'traits')

In [103]:
colnames(coloc_signal_summarized_eqtl) = paste0('egene_', colnames(coloc_signal_summarized_eqtl))

In [104]:
head(coloc_signal_summarized_eqtl,2)

,egene_identifier,egene_high_coloc_signal,egene_intermediate_coloc_signal,egene_low_coloc_signal
,<chr>,<chr>,<chr>,<chr>
1,AFF3,RA_OKADA_TRANS_ETHNIC,NA,NA
2,AP1S3,NA,TAGC_Asthma_EUR,NA


In [105]:
length(unique(coloc_signal_summarized_eqtl$egene_identifier))

[1] 45

In [106]:
### Generate a dataset that contains all PP.H4 values per egene (not dependent on max probability or min probability for H4)

In [107]:
head(eqtl_summary_filtered,2)

trait,identifier,parameter,value.x,gene.x,overlapping_snps,gene.y,value.y,value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
Astle_et_al_2016_Eosinophil_counts,ABI1,PP.H3.abf,0.9539714,ABI1,271,ABI1,0.9539714,0.9539714
Astle_et_al_2016_Eosinophil_counts,ABRACL,PP.H2.abf,0.7040967,ABRACL,332,ABRACL,0.7040967,0.7040967


In [108]:
eqtl_summary_filtered_H4_all = eqtl_summary[eqtl_summary$parameter == 'PP.H4.abf',]

In [109]:
length(unique(eqtl_summary_filtered_H4_all$identifier[eqtl_summary_filtered_H4_all$value  > 0.5]))

[1] 45

In [110]:
## rename value columns

In [111]:
eqtl_summary_filtered_H4_all$coloc_H4_eGene = eqtl_summary_filtered_H4_all$value

## For Co-EQTLs

### Prepare general sumamries

In [283]:
## Prepare data

In [284]:
head(coeqtl_summary,2)

parameter,value,trait,identifier
<chr>,<dbl>,<chr>,<chr>
nsnps,9.840000e+02,White blood cell count,CELF2_EIF5A
PP.H0.abf,4.485855e-07,White blood cell count,CELF2_EIF5A


In [285]:
## extract amount of SNPs info

In [286]:
n_snps = coeqtl_summary[coeqtl_summary$parameter == 'nsnps',]

In [287]:
n_snps$overlapping_snps = n_snps$value
n_snps$parameter = NULL
n_snps$value = NULL

In [288]:
## Keep all the hypothesis columns

In [289]:
coeqtl_summary = coeqtl_summary[coeqtl_summary$parameter != 'nsnps',]

In [290]:
coeqtl_summary$cell_type = ''

In [291]:
#tail(coeqtl_summary,2)

In [292]:
unique(coeqtl_summary$cell_type)

[1] ""

In [293]:
### Add number of snps to hypothesis columns

In [294]:
head(n_snps,2)

trait,identifier,overlapping_snps
<chr>,<chr>,<dbl>
White blood cell count,CELF2_EIF5A,984
White blood cell count,RPS12_SLF2,1420


In [295]:
nrow(coeqtl_summary)

[1] 145495

In [296]:
head(coeqtl_summary,2)

parameter,value,trait,identifier,cell_type
<chr>,<dbl>,<chr>,<chr>,<chr>
PP.H0.abf,4.485855e-07,White blood cell count,CELF2_EIF5A,
PP.H1.abf,6.078679e-09,White blood cell count,CELF2_EIF5A,


In [297]:
### Add amount of snps to coeqtl_summary

In [298]:
coeqtl_summary = merge(coeqtl_summary, n_snps, by.x = c('trait', 'identifier'), by.y = c('trait', 'identifier'))

In [299]:
nrow(coeqtl_summary)

[1] 145495

In [300]:
head(coeqtl_summary,2)

trait,identifier,parameter,value,cell_type,overlapping_snps
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
Asthma,AAK1_EIF5A,PP.H0.abf,0.89252110,,431
Asthma,AAK1_EIF5A,PP.H1.abf,0.01097482,,431


In [301]:
### Per identifier (= gene_pair) and trait get the hypothesis with maximum probability

In [302]:
max_value = coeqtl_summary %>% group_by(trait, identifier, overlapping_snps) %>% summarise(value = max(value))

`summarise()` has grouped output by 'trait', 'identifier'. You can override
using the `.groups` argument.


In [303]:
head(max_value,2)

trait,identifier,overlapping_snps,value
<chr>,<chr>,<dbl>,<dbl>
Asthma,AAK1_EIF5A,431,0.8925211
Asthma,ABI1_YWHAQ,458,0.5196723


In [304]:
### Filter on the hyptothesis with max probability

In [305]:
coeqtl_summary_filtered = merge(coeqtl_summary, max_value)

In [306]:
coeqtl_summary_filtered = coeqtl_summary_filtered[coeqtl_summary_filtered$value.x == coeqtl_summary_filtered$value.y,]

In [307]:
head(coeqtl_summary_filtered,2)

trait,identifier,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
Asthma,AAK1_EIF5A,PP.H0.abf,0.8925211,,431,431,0.8925211
Asthma,ABI1_YWHAQ,PP.H0.abf,0.5196723,,458,458,0.5196723


In [308]:
coeqtl_summary_filtered$value = coeqtl_summary_filtered$value.x
coeqtl_summary_filtered$overlapping_snps = coeqtl_summary_filtered$overlapping_snps.x

In [309]:
head(coeqtl_summary_filtered,2)

trait,identifier,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Asthma,AAK1_EIF5A,PP.H0.abf,0.8925211,,431,431,0.8925211,0.8925211,431
Asthma,ABI1_YWHAQ,PP.H0.abf,0.5196723,,458,458,0.5196723,0.5196723,458


In [310]:
## Filter on the cases where max probability is for H4

In [311]:
coeqtl_summary_filtered_H4 = coeqtl_summary_filtered[coeqtl_summary_filtered$parameter == 'PP.H4.abf',]

In [312]:
nrow(coeqtl_summary_filtered_H4)

[1] 1851

In [313]:
coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'RPS26_SMDT1',]

trait,identifier,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Asthma,RPS26_SMDT1,PP.H4.abf,0.8092893,,239,239,0.8092893,0.8092893,239
Rheumatoid Arthritis,RPS26_SMDT1,PP.H4.abf,0.9999096,,392,392,0.9999096,0.9999096,392
Type_1_Diabetes,RPS26_SMDT1,PP.H4.abf,1.0000000,,828,828,1.0000000,1.0000000,828


In [314]:
#coeqtl_summary_filtered$co_egene = ''
#coeqtl_summary_filtered$co_egene[coeqtl_summary_filtered$egene.x == coeqtl_summary_filtered$gene1] = coeqtl_summary_filtered$gene2[coeqtl_summary_filtered$egene.x == coeqtl_summary_filtered$gene1]
#coeqtl_summary_filtered$co_egene[coeqtl_summary_filtered$egene.x == coeqtl_summary_filtered$gene2] = coeqtl_summary_filtered$gene1[coeqtl_summary_filtered$egene.x == coeqtl_summary_filtered$gene2]

In [315]:
nrow(coeqtl_summary_filtered)

[1] 29099

In [316]:
head(coeqtl_summary_filtered_H4,2)

trait,identifier,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Asthma,ANXA1_HLA-DRB1,PP.H4.abf,0.6526146,,1150,1150,0.6526146,0.6526146,1150
Asthma,CNTRL_WDR36,PP.H4.abf,0.7887180,,474,474,0.7887180,0.7887180,474


In [317]:
### Match the egene where possible (add the snp with the maximum probability of detailled coloc output)

In [318]:
coeqtl_summary_filtered_H4 = merge(coeqtl_summary_filtered_H4, max_prob_per_ident, by.x = c('identifier', 'trait'), by.y = c('identifier', 'trait'))   # removes some with max H4 indication where prob < 0.5

In [319]:
nrow(coeqtl_summary_filtered_H4 )

[1] 1815

In [320]:
coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'RPS26_SMDT1',]

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
RPS26_SMDT1,Asthma,PP.H4.abf,0.8092893,,239,239,0.8092893,0.8092893,239,12:56007301:G:A,56007301
RPS26_SMDT1,Rheumatoid Arthritis,PP.H4.abf,0.9999096,,392,392,0.9999096,0.9999096,392,12:56007301:G:A,56007301
RPS26_SMDT1,Type_1_Diabetes,PP.H4.abf,1.0000000,,828,828,1.0000000,1.0000000,828,12:56007301:G:A,56007301


In [321]:
#coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'ANXA1_SESN1',]

In [322]:
head(coeqtl_summary_filtered_H4,2)

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ACTB_HLA-A,Multiple Sclerosis,PP.H4.abf,0.9548720,,88,88,0.9548720,0.9548720,88,6:29766321:G:A,29766321
ADK_HLA-DQB1,Inflammatory Bowel Disease,PP.H4.abf,0.5694439,,3333,3333,0.5694439,0.5694439,3333,6:32636745:C:A,32636745


In [323]:
length(unique(coeqtl_summary_filtered_H4$identifier))   # amount of gene-pairs with highest indication of coloc

[1] 705

In [324]:
### Add the information abut the egene ( the tested SNPs min and max position)

In [325]:
head(feature_id_egene,2)

feature_id,egene,co_egene,min_pos,max_pos
<chr>,<chr>,<chr>,<chr>,<chr>
AAK1_EIF5A,EIF5A,AAK1,7304878,7304878
ABI1_YWHAQ,YWHAQ,ABI1,9704049,9704049


In [326]:
nrow(feature_id_egene)

[1] 4170

In [327]:
length(unique(feature_id_egene$feature_id))  # not many duplicates

[1] 4157

In [328]:
head(feature_id_egene %>% group_by(feature_id) %>% dplyr::count() %>% filter(n > 1),3)  # cases with multiple possible egenes

feature_id,n
<chr>,<int>
AIF1_LGALS3,2
ANK3_SSH2,2
ATP10A_FAM13A,2


In [329]:
coeqtl_summary_filtered_H4 = merge(coeqtl_summary_filtered_H4, feature_id_egene, by.x = c('identifier'), by.y = c('feature_id'))

In [330]:
#coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'ANXA1_SESN1',]

In [331]:
length(unique(coeqtl_summary_filtered_H4$identifier))

[1] 705

In [332]:
nrow(coeqtl_summary_filtered_H4)

[1] 1827

In [333]:
### for those examples the egene and co-egene assignment needs to be checked
head(coeqtl_summary_filtered_H4 %>% group_by(identifier, trait) %>% dplyr::count() %>% filter(n> 1))

identifier,trait,n
<chr>,<chr>,<int>
RPS26_SH3YL1,Asthma,2
RPS26_SH3YL1,Rheumatoid Arthritis,2
RPS26_SH3YL1,Type_1_Diabetes,2
RPS26_SMDT1,Asthma,2
RPS26_SMDT1,Rheumatoid Arthritis,2
RPS26_SMDT1,Type_1_Diabetes,2


In [334]:
unique(coeqtl_summary_filtered_H4[(coeqtl_summary_filtered_H4$identifier == 'RPS26_VAMP2') & (coeqtl_summary_filtered_H4$trait == 'Asthma'),])   # example

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
RPS26_VAMP2,Asthma,PP.H4.abf,0.8094592,,367,367,0.8094592,0.8094592,367,12:56007301:G:A,56007301,RPS26,VAMP2,56007301,56007301
RPS26_VAMP2,Asthma,PP.H4.abf,0.8094592,,367,367,0.8094592,0.8094592,367,12:56007301:G:A,56007301,VAMP2,RPS26,8233498,8233498


In [335]:
#### Calculate distance ( to identify which one is the egen)

In [336]:
coeqtl_summary_filtered_H4$position_max_coloc_snp = str_extract(coeqtl_summary_filtered_H4$snp, ':\\d.*\\d:')
coeqtl_summary_filtered_H4$position_max_coloc_snp  = str_replace_all(coeqtl_summary_filtered_H4$position_max_coloc_snp, ':', '')

In [337]:
unique(coeqtl_summary_filtered_H4[(coeqtl_summary_filtered_H4$identifier == 'RPS26_VAMP2') & (coeqtl_summary_filtered_H4$trait == 'Asthma'),])   # example

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
RPS26_VAMP2,Asthma,PP.H4.abf,0.8094592,,367,367,0.8094592,0.8094592,367,12:56007301:G:A,56007301,RPS26,VAMP2,56007301,56007301,56007301
RPS26_VAMP2,Asthma,PP.H4.abf,0.8094592,,367,367,0.8094592,0.8094592,367,12:56007301:G:A,56007301,VAMP2,RPS26,8233498,8233498,56007301


In [338]:
## Calculate distance
coeqtl_summary_filtered_H4$diff_min = as.numeric(coeqtl_summary_filtered_H4$position_max_coloc_snp) - as.numeric(coeqtl_summary_filtered_H4$min_pos)

In [339]:
coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'RPS26_SMDT1',]

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
RPS26_SMDT1,Asthma,PP.H4.abf,0.8092893,,239,239,0.8092893,0.8092893,239,12:56007301:G:A,56007301,RPS26,SMDT1,56007301,56007301,56007301,0
RPS26_SMDT1,Asthma,PP.H4.abf,0.8092893,,239,239,0.8092893,0.8092893,239,12:56007301:G:A,56007301,SMDT1,RPS26,42067810,42108675,56007301,13939491
RPS26_SMDT1,Rheumatoid Arthritis,PP.H4.abf,0.9999096,,392,392,0.9999096,0.9999096,392,12:56007301:G:A,56007301,RPS26,SMDT1,56007301,56007301,56007301,0
RPS26_SMDT1,Rheumatoid Arthritis,PP.H4.abf,0.9999096,,392,392,0.9999096,0.9999096,392,12:56007301:G:A,56007301,SMDT1,RPS26,42067810,42108675,56007301,13939491
RPS26_SMDT1,Type_1_Diabetes,PP.H4.abf,1.0000000,,828,828,1.0000000,1.0000000,828,12:56007301:G:A,56007301,RPS26,SMDT1,56007301,56007301,56007301,0
RPS26_SMDT1,Type_1_Diabetes,PP.H4.abf,1.0000000,,828,828,1.0000000,1.0000000,828,12:56007301:G:A,56007301,SMDT1,RPS26,42067810,42108675,56007301,13939491


In [340]:
coeqtl_summary_filtered_H4 = coeqtl_summary_filtered_H4 %>% group_by(identifier,trait) %>% filter(abs(diff_min) == min(abs(diff_min)))

In [341]:
#coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'ANXA1_SESN1',]

In [342]:
#coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$identifier == 'ANXA1_SESN1',]

In [343]:
unique(coeqtl_summary_filtered_H4[(coeqtl_summary_filtered_H4$identifier == 'RPS26_SMDT1') & (coeqtl_summary_filtered_H4$trait == 'Rheumatoid Arthritis'),])   # example

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
RPS26_SMDT1,Rheumatoid Arthritis,PP.H4.abf,0.9999096,,392,392,0.9999096,0.9999096,392,12:56007301:G:A,56007301,RPS26,SMDT1,56007301,56007301,56007301,0


In [344]:
max(abs(coeqtl_summary_filtered_H4$diff_min))

[1] 5898893

In [345]:
head(coeqtl_summary_filtered_H4 %>% arrange(-abs(diff_min)),5)   # TBD: whether it is possible that those could be multiple signals

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
IL32_PZP,Type_1_Diabetes,PP.H4.abf,0.5484011,,1418,1418,0.5484011,0.5484011,1418,12:8943153:T:C,8943153,IL32,PZP,3044260,3044260,8943153,5898893
RPL11_USP53,White blood cell count,PP.H4.abf,0.5247751,,1784,1784,0.5247751,0.5247751,1784,4:119635859:G:T,119635859,USP53,RPL11,119160318,119160318,119635859,475541
RPL12_USP53,White blood cell count,PP.H4.abf,0.5456824,,1154,1154,0.5456824,0.5456824,1154,4:119595442:T:C,119595442,USP53,RPL12,119160318,119160318,119595442,435124
RPL37A_USP53,White blood cell count,PP.H4.abf,0.5096204,,1340,1340,0.5096204,0.5096204,1340,4:119595442:T:C,119595442,USP53,RPL37A,119160318,119160318,119595442,435124
RPS12_USP53,White blood cell count,PP.H4.abf,0.5107535,,1676,1676,0.5107535,0.5107535,1676,4:119595442:T:C,119595442,USP53,RPS12,119160318,119160318,119595442,435124


In [346]:
### for those examples the egene and co-egene assignment needs to be checked
#head(coeqtl_summary_filtered_H4 %>% group_by(identifier, trait) %>% dplyr::count() %>% filter(n> 1))

In [347]:
#### Catagorize coloc signal (per gene-pair and egene)

In [348]:
head(coeqtl_summary_filtered_H4,2)

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
ACTB_HLA-A,Multiple Sclerosis,PP.H4.abf,0.9548720,,88,88,0.9548720,0.9548720,88,6:29766321:G:A,29766321,HLA-A,ACTB,29948728,29955726,29766321,-182407
ADK_HLA-DQB1,Inflammatory Bowel Disease,PP.H4.abf,0.5694439,,3333,3333,0.5694439,0.5694439,3333,6:32636745:C:A,32636745,HLA-DQB1,ADK,32658212,32658212,32636745,-21467


In [349]:
### Filter on probability greater 0.5
coeqtl_summary_filtered_H4 = coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$value > 0.5,]

In [350]:
length(unique(coeqtl_summary_filtered_H4$egene))

[1] 30

In [351]:
### categorize coloc signal
coeqtl_summary_filtered_H4$coloc_cat = 'low_coloc_signal'
coeqtl_summary_filtered_H4$coloc_cat[coeqtl_summary_filtered_H4$value > 0.75] = 'intermediate_coloc_signal'
coeqtl_summary_filtered_H4$coloc_cat[coeqtl_summary_filtered_H4$value > 0.9] = 'high_coloc_signal'

In [352]:
coloc_signal_summarized = coeqtl_summary_filtered_H4 %>% group_by(identifier, egene, co_egene, coloc_cat) %>% summarise(traits = paste0(trait, collapse = ';'))

`summarise()` has grouped output by 'identifier', 'egene', 'co_egene'. You can
override using the `.groups` argument.


In [353]:
coloc_signal_summarized = coloc_signal_summarized %>% dcast(identifier + egene + co_egene ~ coloc_cat, value.var = 'traits')

In [354]:
head(coloc_signal_summarized,2)

,identifier,egene,co_egene,high_coloc_signal,intermediate_coloc_signal,low_coloc_signal
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ACTB_HLA-A,HLA-A,ACTB,Multiple Sclerosis,NA,NA
2,ADK_HLA-DQB1,HLA-DQB1,ADK,NA,NA,Inflammatory Bowel Disease


## Analyze overlap between eQTL and co-eQTL coloc signal

In [184]:
head(coeqtl_summary_filtered_H4,2)

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min,coloc_cat
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
ABI1_RPS26,RA_OKADA_TRANS_ETHNIC,PP.H4.abf,0.9998975,,386,386,0.9998975,0.9998975,386,12:56007301:G:A,56007301,RPS26,ABI1,56007301,56007301,56007301,0,high_coloc_signal
ABI1_RPS26,TAGC_Asthma_EUR,PP.H4.abf,0.8102639,,206,206,0.8102639,0.8102639,206,12:56007301:G:A,56007301,RPS26,ABI1,56007301,56007301,56007301,0,intermediate_coloc_signal


In [185]:
head(eqtl_summary_filtered_H4,2)

trait,identifier,parameter,value.x,gene.x,overlapping_snps,gene.y,value.y,value,coloc_cat
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
Astle_et_al_2016_Eosinophil_counts,COX4I1,PP.H4.abf,0.6421644,COX4I1,354,COX4I1,0.6421644,0.6421644,low_coloc_signal
Astle_et_al_2016_Eosinophil_counts,MDM4,PP.H4.abf,0.7205195,MDM4,388,MDM4,0.7205195,0.7205195,low_coloc_signal


In [186]:
head(sign_coeqtls,2)

,X,feature_id,snp_id,beta,p_value,q_val_filter,egene,co_egene,pos
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,0,RPS26_SLC25A3,12:56007301:G:A,-0.1618779,6.399083e-07,0.020452420,RPS26,SLC25A3,56007301
2,1,RPS14_TOMM7,7:22813024:T:A,0.1671983,9.461872e-08,0.003874102,TOMM7,RPS14,22813024


In [187]:
length(unique(sign_coeqtls$egene))

[1] 103

In [188]:
### Add coloc information to sign co-eqtls

In [189]:
data.frame(trait = unique(eqtl_summary_filtered_H4$trait))

trait
<chr>
Astle_et_al_2016_Eosinophil_counts
Astle_et_al_2016_High_light_scatter_reticulocyte_count
Astle_et_al_2016_Lymphocyte_counts
Astle_et_al_2016_Reticulocyte_count
Astle_et_al_2016_Sum_eosinophil_basophil_counts
Astle_et_al_2016_White_blood_cell_count
BCAC_ER_positive_BreastCancer_EUR
GIANT_2017_BMI_Active_EUR
IBD.EUR.Crohns_Disease


In [190]:
nrow(sign_coeqtls)

[1] 1430

In [191]:
coloc_eval = merge(sign_coeqtls[,c('egene', 'feature_id', 'co_egene', 'snp_id')], data.frame(trait = unique(eqtl_summary_filtered_H4$trait))) ## add all the tested traits as dummies

In [192]:
#coloc_eval[coloc_eval$feature_id == 'RPS26_SMDT1',]

In [193]:
nrow(coloc_eval)

[1] 31460

In [194]:
head(coloc_eval,2)

,egene,feature_id,co_egene,snp_id,trait
,<chr>,<chr>,<chr>,<chr>,<chr>
1,RPS26,RPS26_SLC25A3,SLC25A3,12:56007301:G:A,Astle_et_al_2016_Eosinophil_counts
2,TOMM7,RPS14_TOMM7,RPS14,7:22813024:T:A,Astle_et_al_2016_Eosinophil_counts


In [195]:
#head(coloc_eval,2)

In [196]:
#head(eqtl_summary_filtered_H4,2)

In [197]:
coloc_eval = merge(coloc_eval, eqtl_summary_filtered_H4[,c('trait', 'identifier', 'value', 'coloc_cat')],
                   by.x = c('egene', 'trait'), by.y = c('identifier', 'trait'), all.x = TRUE)

In [198]:
nrow(coloc_eval)

[1] 31460

In [199]:
head(coeqtl_summary_filtered_H4,2)

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min,coloc_cat
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
ABI1_RPS26,RA_OKADA_TRANS_ETHNIC,PP.H4.abf,0.9998975,,386,386,0.9998975,0.9998975,386,12:56007301:G:A,56007301,RPS26,ABI1,56007301,56007301,56007301,0,high_coloc_signal
ABI1_RPS26,TAGC_Asthma_EUR,PP.H4.abf,0.8102639,,206,206,0.8102639,0.8102639,206,12:56007301:G:A,56007301,RPS26,ABI1,56007301,56007301,56007301,0,intermediate_coloc_signal


In [200]:
#head(coloc_eval[coloc_eval$feature_id == 'RPS26_SMDT1',],10)

In [201]:
#head(coeqtl_summary_filtered_H4,2)

In [202]:
#head(coeqtl_summary_filtered_H4[!is.na(str_extract(coeqtl_summary_filtered_H4$identifier , 'SMDT1')),],2)

In [203]:
coloc_eval = merge(coloc_eval, coeqtl_summary_filtered_H4[,c('trait', 'identifier', 'egene', 'value', 'coloc_cat')], 
                   by.x = c('feature_id', 'trait', 'egene'), by.y = c('identifier', 'trait', 'egene'), all.x = TRUE)

In [204]:
nrow(coloc_eval)

[1] 31460

In [205]:
#coloc_eval[(coloc_eval$egene == 'SMDT1') & (!is.na(coloc_eval$value.y)),]

In [206]:
head(coloc_eval,2) ## x = indication for eqtl; y = indication for co-eqtl

,feature_id,trait,egene,co_egene,snp_id,value.x,coloc_cat.x,value.y,coloc_cat.y
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>
1,ABI1_RPS26,Astle_et_al_2016_Eosinophil_counts,RPS26,ABI1,12:56007301:G:A,1,high_coloc_signal,NA,NA
2,ABI1_RPS26,Astle_et_al_2016_High_light_scatter_reticulocyte_count,RPS26,ABI1,12:56007301:G:A,NA,NA,NA,NA


In [207]:
### Evaluate per triplet
coloc_eval %>% group_by(trait) %>% summarise(amount_no_coloc_coeqtl = sum(is.na(value.x) & is.na(value.y)),
                                             amount_coloc_coeqtl_matches = sum(!is.na(value.x) & !is.na(value.y)),
                                             amount_coloc_coeqtl_only_coeqtl = sum(is.na(value.x) & (!is.na(value.y))),
                                             amount_coloc_coeqtl_only_eqtl = sum(!is.na(value.x) & (is.na(value.y))),
                                             
                                        )
                                            
                                             
                                             

trait,amount_no_coloc_coeqtl,amount_coloc_coeqtl_matches,amount_coloc_coeqtl_only_coeqtl,amount_coloc_coeqtl_only_eqtl
<chr>,<int>,<int>,<int>,<int>
Astle_et_al_2016_Eosinophil_counts,979,413,3,35
Astle_et_al_2016_High_light_scatter_reticulocyte_count,1346,65,1,18
Astle_et_al_2016_Lymphocyte_counts,1345,76,1,8
Astle_et_al_2016_Reticulocyte_count,1331,66,16,17
Astle_et_al_2016_Sum_eosinophil_basophil_counts,982,407,3,38
Astle_et_al_2016_White_blood_cell_count,1426,4,0,0
BCAC_ER_positive_BreastCancer_EUR,1427,3,0,0
GIANT_2017_BMI_Active_EUR,1414,0,0,16
IBD.EUR.Crohns_Disease,1428,2,0,0


In [208]:
coloc_eval$value.x[is.na(coloc_eval$value.x)] = 0
coloc_eval$value.y[is.na(coloc_eval$value.y)] = 0

In [209]:
coloc_eval = coloc_eval %>% group_by(trait, egene) %>% summarise(value.x = max(value.x), value.y = max(value.y)) ## summairze coloc signal over egenes

`summarise()` has grouped output by 'trait'. You can override using the
`.groups` argument.


In [210]:
### Evaluate per egene
coloc_eval %>% group_by(trait) %>% summarise(amount_egene = length(unique(egene)),
                                             amount_no_coloc_egene = length(unique(egene[(value.x == 0) & (value.y == 0)])),
                                             amount_coloc_egene_matches = length(unique(egene[(value.x!= 0) & (value.y != 0 )])),
                                             amount_coloc_egene_only_coeqtl = length(unique(egene[(value.x == 0) & (value.y != 0)])),
                                             amount_coloc_egene_only_eqtl = length(unique(egene[(value.x != 0) & (value.y == 0)])))

trait,amount_egene,amount_no_coloc_egene,amount_coloc_egene_matches,amount_coloc_egene_only_coeqtl,amount_coloc_egene_only_eqtl
<chr>,<int>,<int>,<int>,<int>,<int>
Astle_et_al_2016_Eosinophil_counts,103,98,3,2,0
Astle_et_al_2016_High_light_scatter_reticulocyte_count,103,101,1,1,0
Astle_et_al_2016_Lymphocyte_counts,103,100,2,1,0
Astle_et_al_2016_Reticulocyte_count,103,99,1,3,0
Astle_et_al_2016_Sum_eosinophil_basophil_counts,103,99,1,2,1
Astle_et_al_2016_White_blood_cell_count,103,101,2,0,0
BCAC_ER_positive_BreastCancer_EUR,103,102,1,0,0
GIANT_2017_BMI_Active_EUR,103,102,0,0,1
IBD.EUR.Crohns_Disease,103,102,1,0,0


In [211]:
## Summarise across traits

In [212]:
coloc_eval = coloc_eval %>% group_by( egene) %>% summarise(value.x = max(value.x), value.y = max(value.y)) ## summairze coloc signal over egenes

In [213]:
### Evaluate per egene
coloc_eval %>% summarise(amount_egene = length(unique(egene)),
                                             amount_no_coloc_egene = length(unique(egene[(value.x == 0) & (value.y == 0)])),
                                             amount_coloc_egene_matches = length(unique(egene[(value.x!= 0) & (value.y != 0 )])),
                                             amount_coloc_egene_only_coeqtl = length(unique(egene[(value.x == 0) & (value.y != 0)])),
                                             amount_coloc_egene_only_eqtl = length(unique(egene[(value.x != 0) & (value.y == 0)])))

amount_egene,amount_no_coloc_egene,amount_coloc_egene_matches,amount_coloc_egene_only_coeqtl,amount_coloc_egene_only_eqtl
<int>,<int>,<int>,<int>,<int>
103,82,12,7,2


In [214]:
### Get the egene only signals

In [215]:
unique(coloc_eval$egene[(coloc_eval$value.x == 0 ) & (coloc_eval$value.y !=0)])

[1] "CDK13"      "CYB5A"      "FOXK2"      "HLA-A"      "HLA-DRB1"  
[6] "NACA_RPS26" "RIOK3"

## Extract the colocalizing signals and join to meta-info (eqtl and co-eqt)

In [2771]:
### Add information about colocalization to significant co-eqtls

In [2772]:
sign_coeqtls$pos = NULL

In [2773]:
nrow(sign_coeqtls)

[1] 7240

In [2774]:
sign_coeqtls = merge(sign_coeqtls, coloc_signal_summarized, by.x = c('feature_id', 'egene', 'co_egene'), by.y = c('identifier', 'egene', 'co_egene'), all.x = TRUE)

In [2775]:
head(sign_coeqtls,2)

,feature_id,egene,co_egene,X,snp_id,beta,p_value,q_val_filter,high_coloc_signal,intermediate_coloc_signal,low_coloc_signal
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,AAK1_EIF5A,EIF5A,AAK1,0,17:7304878:C:T,-0.2303832,1.020164e-13,1.095937e-09,NA,NA,NA
2,ABI1_YWHAQ,YWHAQ,ABI1,17,2:9704049:A:G,-0.1376059,1.063316e-05,2.693636e-02,NA,NA,NA


In [2776]:
nrow(sign_coeqtls)

[1] 7240

In [2777]:
#### Join to eqtl coloc information

In [2778]:
length(unique(coloc_signal_summarized_eqtl$egene_identifier))

[1] 61

In [2779]:
sign_coeqtls = merge(sign_coeqtls, coloc_signal_summarized_eqtl, by.x = 'egene', by.y = 'egene_identifier', all.x = TRUE)

In [2780]:
nrow(sign_coeqtls)

[1] 7240

In [2781]:
head(sign_coeqtls,2)

,egene,feature_id,co_egene,X,snp_id,beta,p_value,q_val_filter,high_coloc_signal,intermediate_coloc_signal,low_coloc_signal,egene_high_coloc_signal,egene_intermediate_coloc_signal,egene_low_coloc_signal
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ABRACL,ABRACL_HOTAIRM1,HOTAIRM1,28,6:139029769:A:G,0.1469621,1.714333e-05,0.03940812,NA,NA,NA,NA,NA,NA
2,ACP1,ACP1_FAM153B,FAM153B,131,2:217826:G:C,0.1394655,2.229418e-05,0.04851411,NA,NA,NA,NA,NA,NA


In [2782]:
write.csv(sign_coeqtls, paste0(result_path, '/G2/Coloc_Meta_', dataset, '_', cell_type_var, '.csv'))

In [2783]:
#### Investigate amount of overlapping and non overlapping colocalization signals

In [2784]:
### Amount of colocalization signals for egenes

In [2785]:
length(unique(sign_coeqtls$egene[(!is.na(sign_coeqtls$egene_high_coloc_signal)) | (!is.na(sign_coeqtls$egene_intermediate_coloc_signal)) | (!is.na(sign_coeqtls$egene_low_coloc_signal))]))

[1] 21

In [2786]:
### Amount of colocalization signals for co-eGenes

In [2787]:
length(unique(sign_coeqtls$egene[(!is.na(sign_coeqtls$high_coloc_signal)) | (!is.na(sign_coeqtls$intermediate_coloc_signal)) | (!is.na(sign_coeqtls$low_coloc_signal))]))

[1] 30

In [2788]:
### Investigate overlapping signal (! not specifically checked per trait here)
length(unique(sign_coeqtls$egene[((!is.na(sign_coeqtls$high_coloc_signal)) | (!is.na(sign_coeqtls$intermediate_coloc_signal)) | (!is.na(sign_coeqtls$low_coloc_signal))) &
                                ((!is.na(sign_coeqtls$egene_high_coloc_signal)) | (!is.na(sign_coeqtls$egene_intermediate_coloc_signal)) | (!is.na(sign_coeqtls$egene_low_coloc_signal)))
                                
                                ]))


[1] 14

In [2789]:
### Investigate co-eqtl signal but no eqtl 

In [2790]:
length(unique(sign_coeqtls$egene[((!is.na(sign_coeqtls$high_coloc_signal)) | (!is.na(sign_coeqtls$intermediate_coloc_signal)) | (!is.na(sign_coeqtls$low_coloc_signal))) &
                                ((is.na(sign_coeqtls$egene_high_coloc_signal)) & (is.na(sign_coeqtls$egene_intermediate_coloc_signal)) & (is.na(sign_coeqtls$egene_low_coloc_signal)))
                                
                                ]))

[1] 16

In [2791]:
unique(sign_coeqtls$egene[((!is.na(sign_coeqtls$high_coloc_signal)) | (!is.na(sign_coeqtls$intermediate_coloc_signal)) | (!is.na(sign_coeqtls$low_coloc_signal))) &
                                ((is.na(sign_coeqtls$egene_high_coloc_signal)) & (is.na(sign_coeqtls$egene_intermediate_coloc_signal)) & (is.na(sign_coeqtls$egene_low_coloc_signal)))
                                
                                ])

[1] "AFF3"       "DIP2A"      "HLA-A"      "HLA-C"      "HLA-DQB1"  
 [6] "IL32"       "IL6R"       "NACA_RPS26" "PRMT2"      "PZP"       
[11] "RBMS1"      "RUNX2"      "SESN1"      "STK38"      "TFDP2"     
[16] "USP53"

In [2792]:
## Trait specific investigation

In [2793]:
nrow(sign_coeqtls)

[1] 7240

In [2794]:
head(sign_coeqtls,2)

,egene,feature_id,co_egene,X,snp_id,beta,p_value,q_val_filter,high_coloc_signal,intermediate_coloc_signal,low_coloc_signal,egene_high_coloc_signal,egene_intermediate_coloc_signal,egene_low_coloc_signal
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ABRACL,ABRACL_HOTAIRM1,HOTAIRM1,28,6:139029769:A:G,0.1469621,1.714333e-05,0.03940812,NA,NA,NA,NA,NA,NA
2,ACP1,ACP1_FAM153B,FAM153B,131,2:217826:G:C,0.1394655,2.229418e-05,0.04851411,NA,NA,NA,NA,NA,NA


## Summarize amount of colocalization signals per eGene

In [2795]:
### Summarize per eGene and Trait

In [2796]:
head(coeqtl_summary_filtered_H4,2)

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min,coloc_cat
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
ACTB_HLA-A,Multiple Sclerosis,PP.H4.abf,0.9548720,,88,88,0.9548720,0.9548720,88,6:29766321:G:A,29766321,HLA-A,ACTB,29948728,29955726,29766321,-182407,high_coloc_signal
ADK_HLA-DQB1,Inflammatory Bowel Disease,PP.H4.abf,0.5694439,,3333,3333,0.5694439,0.5694439,3333,6:32636745:C:A,32636745,HLA-DQB1,ADK,32658212,32658212,32636745,-21467,low_coloc_signal


In [2797]:
head(coeqtl_summary_filtered_H4[coeqtl_summary_filtered_H4$egene == 'SMDT1',])

identifier,trait,parameter,value.x,cell_type,overlapping_snps.x,overlapping_snps.y,value.y,value,overlapping_snps,snp,position,egene,co_egene,min_pos,max_pos,position_max_coloc_snp,diff_min,coloc_cat
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>


In [2798]:
overview_h_amounts = coeqtl_summary_filtered_H4 %>% group_by(parameter, trait, egene ) %>% summarise(n = n(), 
                                                                                                    mean_value = mean(value), 
                                                                                                    amount_greater_0.9 = sum(value > 0.9),  
                                                                                                    amount_greater_0.75 = sum(value > 0.75),
                                                                                                    amount_greater_0.5 = sum(value > 0.5),  
                                                                                                    max_overlap_snps = max(overlapping_snps), 
                                                                                                    min_overlap_snps = min(overlapping_snps), 
                                                                                                    mean_overlap_snps = mean(overlapping_snps),
                                                                                                   co_egenes = paste0(co_egene, collapse = ','))


# previously:  coeqtl_summary_filtered; now only H4 evluation

`summarise()` has grouped output by 'parameter', 'trait'. You can override
using the `.groups` argument.


In [2799]:
overview_h_amounts = overview_h_amounts[order(overview_h_amounts$trait),]

In [2800]:
nrow(overview_h_amounts )

[1] 46

In [2801]:
overview_h_amounts = merge(overview_h_amounts , eqtl_summary_filtered_H4_all[,c('trait', 'identifier', 'coloc_H4_eGene')], by.x = c('trait', 'egene'), by.y = c('trait', 'identifier'), all.x = TRUE)

In [2802]:
head(overview_h_amounts %>% arrange(-n),20)

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,co_egenes,coloc_H4_eGene
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,Type_1_Diabetes,RPS26,PP.H4.abf,566,0.9962146,566,566,566,2246,408,1016.7120,"EEF1G,FAU,PTMA,RPL10A,RPL11,RPL12,RPL13,RPL14,RPL23A,RPL27A,RPL28,RPL31,RPL32,RPL36,RPL37A,RPL3,RPL4,RPL7A,RPL7,RPL9,RPLP0,RPLP1,RPLP2,RPS10,RPS12,RPS14,RPS15,RPS16,RPS17,RPS23,RPS24,RPS25,RPS27L,RPS6KA1,RPS6KA2,RPS6KA5,RPTOR,RRAS2,RRBP1,RREB1,RRM1,RRP12,RSF1,RSL1D1,RSL24D1,RSRC1,RSRP1,RSU1,RTCA,RTKN2,RTN3,RTN4,RUNX2,RUNX3,RYBP,S100A10,S100A11,S100A6,SAMD3,SAMD9L,SAMHD1,SAMSN1,SAP30BP,SAR1B,SARAF,SAT2,SATB1,SCAF8,SCAMP2,SCFD1,SCGB3A1,SCLT1,SCML4,SCOC,SCP2,SCRN2,SCYL2,SDHA,SDK2,SEC11A,SEC11C,SEC14L1,SEC14L2,SEC23B,SEC24D,SEC62,SECISBP2L,SEL1L3,SELENOH,SELENOI,SELENOM,SELENOW,SELL,SELPLG,SEM1,SEMA4D,SENP1,SENP2,SENP7,SERINC1,SERP1,SERPINB6,SESN3,SETD7,SF3B1,SFMBT2,SFXN1,SGMS1,SH2B3,SH2D2A,SH3BGRL3,SH3BP1,SH3BP5,SH3GL1,SH3GLB1,SH3YL1,SHISA5,SHISAL2A,SHKBP1,SHLD2,SIAH2,SIK3,SIMC1,SIPA1L1,SIRT6,SKP1,SLA,SLAMF1,SLAMF6,SLBP,SLC14A1,SLC16A1,SLC17A5,SLC25A13,SLC25A24,SLC25A25,SLC25A26,SLC25A3,SLC25A32,SLC25A46,SLC2A13,SLC35D1,SLC38A1,SLC38A10,SLC38A2,SLC39A8,SLC40A1,SLC4A7,SLC6A6,SLC7A1,SLC9A1,SLC9A3R1,SLC9A8,SLC9A9,SLCO3A1,SLF1,SLFN12L,SLFN5,SLK,SLMAP,SMAP1,SMARCC1,SMCHD1,SMDT1,SMG5,SMG7,SMIM20,SMIM26,SMIM27,SMURF1,SMYD2,SNED1,SNHG7,SNHG8,SNIP1,SNRPB,SNRPD2,SNRPF,SNRPN,SNTB1,SNU13,SNX10,SNX13,SNX14,SNX19,SNX25,SNX4,SNX9,SOCS2,SOD1,SORBS3,SORL1,SOS1,SP100,SP140,SP3,SP4,SPAG1,SPCS1,SPCS2,SPCS3,SPECC1L,SPEN,SPIDR,SPINT2,SPOCK2,SPON2,SPPL2A,SPPL3,SPTLC2,SPTY2D1,SQOR,SRA1,SREBF2,SRGN,SRI,SRP19,SRSF7,SRSF8,SS18L2,SSBP2,SSBP4,SSH2,SSR3,ST13,ST6GAL1,ST8SIA4,STAG1,STAM,STAT1,STIM1,STIM2,STK10,STK17A,STK39,STK4,STMN3,STRN3,STT3B,STX18-AS1,STX8,STXBP2,STXBP3,SUB1,SUCLG2,SUCO,SUMO1,SUPT3H,SUPT5H,SUPT6H,SURF1,SURF4,SUSD3,SVIP,SYMPK,SYNE1,SYNE2,SYNE3,SYNJ1,SYPL1,SYTL1,SYTL2,TAB2,TACC1,TAF13,TAF1B,TAF2,TAF4B,TAF7,TANC2,TANGO2,TANK,TAOK1,TAOK3,TATDN1,TATDN3,TAX1BP1,TBC1D14,TBC1D22A,TBC1D2B,TBC1D4,TBCD,TBXAS1,TC2N,TCEA3,TCF20,TCF25,TCIRG1,TCOF1,TCP10L,TENT2,TENT4A,TENT4B,TENT5C,TESPA1,TET2,TEX264,TFAP4,TFG,TFRC,TG,TGFB1,TGFBR1,TGFBR2,TGFBR3,THEMIS,THOC1,THRA,TIAM1,TIAM2,TIFA,TIGAR,TIGIT,TIMM10,TIMM13,TIMM23,TIPARP,TIPRL,TLE4,TLK1,TMA7,TMBIM1,TMC6,TMC8,TMCC1,TMED5,TMED9,TMEM106B,TMEM116,TMEM123,TMEM127,TMEM135,TMEM140,TMEM141,TMEM156,TMEM161B,TMEM165,TMEM167A,TMEM181,TMEM203,TMEM204,TMEM243,TMEM245,TMEM256,TMEM45B,TMEM50A,TMEM65,TMEM71,TMEM87B,TMEM9B,TMIGD2,TMOD3,TMX4,TNFAIP3,TNFRSF14,TNFRSF25,TNFRSF4,TNFSF10,TNIK,TNIP1,TNKS2,TNPO3,TNRC6A,TOM1,TOMM7,TOX,TP53BP1,TP53BP2,TPD52,TPGS2,TPI1,TPK1,TPM3,TPP2,TPST2,TRA2A,TRABD,TRABD2A,TRAC,TRAF1,TRAF3,TRAF5,TRAK1,TRANK1,TRAPPC10,TRAPPC4,TRBC1,TRBC2,TRG-AS1,TRIB2,TRIM22,TRIM24,TRIM37,TRIM4,TRMT112,TRPC4AP,TRPS1,TSC1,TSC22D2,TSC22D4,TSEN15,TSGA10,TSHZ2,TSPO,TSTD1,TTC3,TTC39C,TTC39C-AS1,TTC7A,TTF2,TTLL3,TTLL5,TUBA1C,TULP4,TWF2,TXK,TXN,TXN2,TXNDC11,TXNDC15,TXNRD1,TYMP,U2AF1L4,UBA6,UBAP1,UBE2E2,UBE2G1,UBE2H,UBE2I,UBE2J1,UBE2K,UBE2L3,UBE2O,UBE2Q2,UBE2R2,UBE3C,UBL3,UBQLN1,UBR2,UBXN11,UIMC1,ULK4,UMAD1,UPP1,UQCR11,UQCRH,USE1,USF3,USO1,USP12,USP14,USP15,USP34,USP36,USP4,USP42,USP47,USP48,USP7,USP8,UTRN,VAMP2,VAMP8,VASP,VAV3,VCL,VDAC1,VDAC2,VEZT,VIRMA,VKORC1,VNN2,VOPP1,VPS13A,VPS13C,VPS33A,VPS50,VPS53,VPS54,VSIR,VTI1A,VWA8,WAPL,WARS2,WASHC2A,WBP1L,WDFY1,WDR1,WDR11,WDR20,WDR26,WDR37,WDR7,WDR70,WIPF1,WIPI1,WNK1,WSB1,WTAP,WWOX,WWP1,WWP2,XPNPEP1,XPO1,XPO4,XPO6,XYLT1,YAF2,YBEY,YBX1,YEATS2,YEATS4,YWHAB,YWHAE,YWHAG,YWHAH,YWHAQ,YWHAZ,ZAP70,ZBTB1,ZBTB20-AS5,ZBTB38,ZC3H12D,ZC3H6,ZC3H7A,ZC3HAV1,ZCCHC10,ZDHHC20,ZDHHC4,ZFAND3,ZFAND6,ZFAS1,ZFC3H1,ZFP36L1,ZFP36L2,ZFR,ZFYVE28,ZMIZ1,ZMYM2,ZNF10,ZNF101,ZNF131,ZNF292,ZNF302,ZNF331,ZNF335,ZNF410,ZNF438,ZNF439,ZNF451,ZNF483,ZNF490,ZNF563,ZNF600,ZNF611,ZNF791,ZNF800,ZNF831,ZNFX1,ZNHIT1,ZSCAN18,ZSWIM6,ZSWIM7,ZYX,ZZEF1,RPS2",1.000000e+00
2,Rheumatoid Arthritis,RPS26,PP.H4

In [2803]:
nrow(overview_h_amounts )

[1] 46

In [2804]:
### Save the summary information

In [2805]:
write.csv(overview_h_amounts[overview_h_amounts$parameter == 'PP.H4.abf',], paste0(result_path, '/G2/',  dataset, '_', cell_type_var, '_Colocalization_Result.csv'))

# Archive

#### Investigate per co_eGene

In [470]:
overview_h_amounts = coeqtl_summary_filtered %>% group_by(parameter, trait, co_egene ) %>% summarise(n = n(), 
                                                                                                    mean_value = mean(value), 
                                                                                                    amount_greater_0.9 = sum(value > 0.9),  
                                                                                                    amount_greater_0.75 = sum(value > 0.75),
                                                                                                    amount_greater_0.5 = sum(value > 0.5),  
                                                                                                    max_overlap_snps = max(overlapping_snps), 
                                                                                                    min_overlap_snps = min(overlapping_snps), 
                                                                                                    mean_overlap_snps = mean(overlapping_snps),
                                                                                                    egenes = paste0(egene.x, collapse = ','))

ERROR: [1m[33mError[39m in `group_by()`:[22m
[1m[22m[33m![39m Must group by variables found in `.data`.
[31m✖[39m Column `co_egene` is not found.


In [471]:
overview_h_amounts = overview_h_amounts[order(overview_h_amounts$trait),]

In [472]:
head(overview_h_amounts[overview_h_amounts$parameter == 'PP.H4.abf',] %>% arrange(-n),40)

parameter,trait,egene,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,co_egenes
<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>
PP.H4.abf,Rheumatoid Arthritis,RPS26,501,0.9962524,499,499,501,8825,2596,4477.7784,"EEF1G,FAU,PTMA,RPL10A,RPL11,RPL12,RPL13,RPL14,RPL23A,RPL27A,RPL28,RPL32,RPL36,RPL37A,RPL4,RPL7A,RPL7,RPL9,RPLP0,RPLP2,RPS12,RPS14,RPS15,RPS16,RPS17,RPS23,RPS24,RPS27L,RPS6KA1,RPS6KA5,RPTOR,RRAS2,RRM1,RRP12,RSF1,RSL1D1,RSL24D1,RSRC1,RSRP1,RSU1,RTCA,RTN3,RTN4,RUNX2,RUNX3,RYBP,S100A10,S100A11,S100A6,SAMD3,SAMD9L,SAMSN1,SAP30BP,SAR1B,SARAF,SAT2,SATB1,SCAF8,SCAMP2,SCFD1,SCGB3A1,SCLT1,SCML4,SCOC,SCP2,SCRN2,SCYL2,SDHA,SEC11A,SEC11C,SEC14L1,SEC14L2,SEC23B,SEC24D,SEC62,SECISBP2L,SELENOH,SELENOM,SELENOW,SELL,SELPLG,SEM1,SEMA4D,SENP2,SENP7,SERINC1,SERP1,SERPINB6,SESN3,SF3B1,SFMBT2,SFXN1,SGMS1,SH2B3,SH3BGRL3,SH3BP1,SH3BP5,SH3GL1,SH3GLB1,SHISA5,SHISAL2A,SHKBP1,SHLD2,SIAH2,SIK3,SIMC1,SIPA1L1,SKP1,SLA,SLAMF1,SLAMF6,SLBP,SLC14A1,SLC17A5,SLC25A13,SLC25A25,SLC25A26,SLC25A3,SLC25A46,SLC2A13,SLC35D1,SLC38A1,SLC38A10,SLC38A2,SLC39A8,SLC40A1,SLC4A7,SLC7A1,SLC9A3R1,SLC9A8,SLC9A9,SLCO3A1,SLF1,SLFN12L,SLFN5,SLMAP,SMAP1,SMARCC1,SMCHD1,SMG5,SMG7,SMIM26,SMIM27,SMURF1,SMYD2,SNED1,SNHG7,SNIP1,SNRPB,SNRPD2,SNRPF,SNRPN,SNTB1,SNU13,SNX13,SNX14,SNX25,SNX4,SNX9,SOCS2,SOD1,SORL1,SOS1,SP100,SP140,SP3,SP4,SPCS1,SPCS2,SPCS3,SPECC1L,SPEN,SPIDR,SPINT2,SPOCK2,SPON2,SPPL2A,SPPL3,SPTLC2,SPTY2D1,SQOR,SRA1,SREBF2,SRGN,SRI,SRP19,SRSF7,SRSF8,SS18L2,SSBP2,SSBP4,SSH2,ST13,ST6GAL1,ST8SIA4,STAG1,STAM,STIM1,STK10,STK39,STK4,STMN3,STRN3,STT3B,STX18-AS1,STX8,STXBP2,STXBP3,SUB1,SUCLG2,SUCO,SUMO1,SUPT3H,SUPT5H,SURF1,SURF4,SUSD3,SVIP,SYMPK,SYNE1,SYNE2,SYNE3,SYNJ1,SYPL1,SYTL1,SYTL2,TAB2,TACC1,TAF2,TAF4B,TAF7,TANC2,TANGO2,TANK,TAOK1,TAOK3,TATDN1,TATDN3,TAX1BP1,TBC1D14,TBC1D22A,TBC1D2B,TBC1D4,TBCD,TBXAS1,TC2N,TCEA3,TCF20,TCF25,TCIRG1,TCOF1,TCP10L,TENT2,TENT4A,TENT4B,TENT5C,TESPA1,TET2,TEX264,TFG,TFRC,TG,TGFB1,TGFBR1,TGFBR2,TGFBR3,THEMIS,THOC1,TIAM1,TIAM2,TIGAR,TIGIT,TIMM10,TIMM13,TIMM23,TIPARP,TIPRL,TLE4,TLK1,TMA7,TMBIM1,TMC6,TMC8,TMCC1,TMED5,TMED9,TMEM116,TMEM123,TMEM135,TMEM141,TMEM156,TMEM161B,TMEM165,TMEM167A,TMEM181,TMEM203,TMEM204,TMEM243,TMEM245,TMEM256,TMEM45B,TMEM50A,TMEM65,TMEM71,TMEM87B,TMEM9B,TMIGD2,TMOD3,TMX4,TNFRSF25,TNFRSF4,TNFSF10,TNIK,TNIP1,TNKS2,TNRC6A,TOM1,TOMM7,TOX,TP53BP1,TP53BP2,TPD52,TPGS2,TPI1,TPK1,TPM3,TPP2,TPST2,TRA2A,TRABD,TRABD2A,TRAC,TRAF1,TRAF3,TRAF5,TRAK1,TRANK1,TRAPPC10,TRBC1,TRBC2,TRG-AS1,TRIB2,TRIM22,TRIM24,TRIM37,TRIM4,TRMT112,TRPC4AP,TRPS1,TSC22D2,TSEN15,TSHZ2,TSPO,TSTD1,TTC3,TTC39C,TTC39C-AS1,TTC7A,TTLL3,TTLL5,TUBA1C,TULP4,TWF2,TXK,TXN,TXN2,TXNDC11,TXNDC15,TXNRD1,TYMP,U2AF1L4,UBA6,UBE2E2,UBE2G1,UBE2I,UBE2J1,UBE2K,UBE2L3,UBE2O,UBE2Q2,UBE3C,UBL3,UBQLN1,UBR2,UBXN11,UIMC1,ULK4,UMAD1,UPP1,UQCR11,UQCRH,USE1,USF3,USO1,USP12,USP14,USP15,USP36,USP4,USP42,USP47,USP48,USP7,USP8,UTRN,VAMP2,VAMP8,VASP,VAV3,VCL,VDAC1,VDAC2,VEZT,VIRMA,VKORC1,VNN2,VOPP1,VPS13A,VPS13C,VPS33A,VPS50,VPS53,VPS54,VTI1A,VWA8,WAPL,WARS2,WASHC2A,WBP1L,WDFY1,WDR1,WDR11,WDR20,WDR26,WDR37,WDR7,WDR70,WIPF1,WNK1,WSB1,WTAP,WWOX,WWP1,WWP2,XPO1,XPO4,XPO6,XYLT1,YAF2,YBX1,YEATS2,YEATS4,YWHAB,YWHAE,YWHAG,YWHAH,YWHAQ,YWHAZ,ZAP70,ZBTB1,ZBTB20-AS5,ZBTB38,ZC3H12D,ZC3H6,ZC3H7A,ZC3HAV1,ZCCHC10,ZDHHC20,ZDHHC4,ZFAND3,ZFAND6,ZFAS1,ZFC3H1,ZFP36L1,ZFP36L2,ZFR,ZFYVE28,ZMYM2,ZNF10,ZNF101,ZNF131,ZNF292,ZNF302,ZNF331,ZNF410,ZNF451,ZNF483,ZNF490,ZNF563,ZNF600,ZNF791,ZNF800,ZNF831,ZNFX1,ZNHIT1,ZSCAN18,ZSWIM6,ZSWIM7,ZYX,ZZEF1,RPS2"
PP.H4.abf,Asthma,RPS26,499,0.7478555,0,300,499,4278,873,1899.6994,"EEF1G,FAU,PTMA,RPL10A,RPL11,RPL12,RPL13,RPL14,RPL23A,RPL27A,RPL28,RPL31,RPL32,RPL36,RPL37A,RPL3,RPL7A,RPL7,RPL9,RPLP0,RPLP1,RPLP2,RPS10,RPS12,RPS14,RPS15,RPS16,RPS17,RPS23,RPS24,RPS25,RPS27L,RPS6KA1,RPS6KA2,RPS6KA5,RPTOR,RRAS2,RRM1,RRP12,RSF1,RSL24D1,RSRC1,RSRP1,RSU1,RTCA,RTKN2,RTN4,RUNX2,RUNX3,RYBP,S100A10,S100A11,S100A6,SAMD3,SAMD9L,SAMSN1,SAP30BP,SAR1B,SARAF,SAT2,SATB1,SCAF8,SCFD1,SCGB3A1,SCLT1,SCML4,SCOC,SCP2,SCRN2,S

### Get certain co-eqtl examples (RA - RPS26 H4 coegenes)

In [115]:
## 41 co-eGenes for Rheumatoid Arthritis with strong colocalization signal

In [116]:
coeqtl_summary_filtered$gene = str_replace(coeqtl_summary_filtered$gene, '_', '')

In [117]:
nrow(coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ),])

[1] 152

In [118]:
head(coeqtl_summary_filtered)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1,Asthma,B_RPS26___EEF1A1__RPS26,B_RPS26,0.3734713,B,EEF1A1_RPS26,381,PP.H0.abf
2,Asthma,B_RPS26___RPL10__RPS26,B_RPS26,0.6338403,B,RPL10_RPS26,381,PP.H4.abf
3,Asthma,B_RPS26___RPL11__RPS26,B_RPS26,0.3473292,B,RPL11_RPS26,381,PP.H4.abf
4,Asthma,B_RPS26___RPL13__RPS26,B_RPS26,0.5521870,B,RPL13_RPS26,381,PP.H4.abf
5,Asthma,B_RPS26___RPL18__RPS26,B_RPS26,0.5154468,B,RPL18_RPS26,381,PP.H0.abf
6,Asthma,B_RPS26___RPL21__RPS26,B_RPS26,0.5951100,B,RPL21_RPS26,381,PP.H4.abf


In [121]:
coloc_examples_rheomatoid_arthritis = coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ),]

In [122]:
#coloc_examples_rheomatoid_arthritis = coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ) & (coeqtl_summary_filtered$egene == 'CD4T_RPS26' ),]

In [123]:
analyze = coloc_examples_rheomatoid_arthritis %>% group_by( gene) %>% count()

In [124]:
coeqtl_summary_filtered[coeqtl_summary_filtered$gene == 'HLA-DPB1_RPS26',]

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
194,Asthma,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.5988243,CD4T,HLA-DPB1_RPS26,381,PP.H4.abf
550,Asthma,CD8T_RPS26___HLA-DPB1__RPS26,CD8T_RPS26,0.4321920,CD8T,HLA-DPB1_RPS26,381,PP.H0.abf
860,Asthma,monocyte_RPS26___HLA-DPB1__RPS26,monocyte_RPS26,0.6660012,monocyte,HLA-DPB1_RPS26,381,PP.H4.abf
1258,Crohn's Disease,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.8220487,CD4T,HLA-DPB1_RPS26,1112,PP.H2.abf
1614,Crohn's Disease,CD8T_RPS26___HLA-DPB1__RPS26,CD8T_RPS26,0.5654940,CD8T,HLA-DPB1_RPS26,1112,PP.H0.abf
1924,Crohn's Disease,monocyte_RPS26___HLA-DPB1__RPS26,monocyte_RPS26,0.9525225,monocyte,HLA-DPB1_RPS26,1112,PP.H2.abf
2322,Inflammatory Bowel Disease,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.8474188,CD4T,HLA-DPB1_RPS26,1110,PP.H2.abf
2678,Inflammatory Bowel Disease,CD8T_RPS26___HLA-DPB1__RPS26,CD8T_RPS26,0.5830098,CD8T,HLA-DPB1_RPS26,1110,PP.H0.abf
2988,Inflammatory Bowel Disease,monocyte_RPS26___HLA-DPB1__RPS26,monocyte_RPS26,0.9819645,monocyte,HLA-DPB1_RPS26,1110,PP.H2.abf


In [125]:
analyze[order(analyze$n, decreasing = TRUE),]

gene,n
<chr>,<int>
AIF1_RPS26,2
C12orf75_RPS26,2
COX7C_RPS26,2
NACA_RPS26,2
RPL11_RPS26,2
RPL4_RPS26,2
RPS10_RPS26,2
RPS24_RPS26,2
RPS26_TIGIT,2


In [126]:
length(unique(coloc_examples_rheomatoid_arthritis$gene))

[1] 141

In [127]:
nrow(coloc_examples_rheomatoid_arthritis %>% group_by(cell_type, gene) %>% count())

[1] 152

In [128]:
head(coloc_examples_rheomatoid_arthritis[order(coloc_examples_rheomatoid_arthritis$value, decreasing = TRUE),],6)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
5255,Rheumatoid Arthritis,NK_RPS26___RPL23__RPS26,NK_RPS26,0.9777943,NK,RPL23_RPS26,879,PP.H4.abf
4839,Rheumatoid Arthritis,CD8T_RPS26___NACA__RPS26,CD8T_RPS26,0.9713088,CD8T,NACA_RPS26,882,PP.H4.abf
4995,Rheumatoid Arthritis,CD8T_RPS26___RPS26__UBA52,CD8T_RPS26,0.9687585,CD8T,RPS26_UBA52,882,PP.H4.abf
5249,Rheumatoid Arthritis,NK_RPS26___RPL15__RPS26,NK_RPS26,0.9687209,NK,RPL15_RPS26,879,PP.H4.abf
4764,Rheumatoid Arthritis,CD8T_RPS26___DNAJB6__RPS26,CD8T_RPS26,0.9685069,CD8T,DNAJB6_RPS26,882,PP.H4.abf
4428,Rheumatoid Arthritis,CD4T_RPS26___FOXP3__RPS26,CD4T_RPS26,0.9669022,CD4T,FOXP3_RPS26,870,PP.H4.abf


In [129]:
coloc_examples_rheomatoid_arthritis %>% group_by(egene) %>% count()

egene,n
<chr>,<int>
B_RPS26,4
CD4T_RPS26,41
CD8T_RPS26,64
monocyte_RPS26,15
NK_RPS26,28


In [130]:
nrow(coloc_examples_rheomatoid_arthritis)

[1] 152

In [131]:
### Mapp positions for coegenes

In [132]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$gene, 'RPS26', '')

In [133]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$cogene, '__', '')

In [134]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$cogene, '_', '')

In [135]:
unique(coloc_examples_rheomatoid_arthritis$cogene)

[1] "RPL11"       "RPL35A"      "RPL41"       "RPLP2"       "ABHD14B"    
  [6] "ADAM19"      "AIF1"        "AKAP13"      "ATP2B1"      "ATP2B4"     
 [11] "C12orf75"    "CD58"        "CHCHD2"      "COX7C"       "CST7"       
 [16] "CYBA"        "EML4"        "ENTPD1"      "FOXP3"       "FTH1"       
 [21] "GALM"        "GK"          "H3F3A"       "HLA-DPB1"    "HLA-DRA"    
 [26] "IGBP1"       "ISG20"       "LINC00493"   "LSM5"        "MIAT"       
 [31] "MIR4435-1HG" "MT2A"        "PRMT2"       "RGS1"        "RNF19A"     
 [36] "SBDS"        "SERF2"       "TCF7"        "TIGIT"       "TMA7"       
 [41] "TMSB10"      "TNFRSF1B"    "UBB"         "YBX1"        "YWHAB"      
 [46] "ABLIM1"      "ANXA2"       "APOBEC3G"    "BHLHE40"     "C12orf57"   
 [51] "C1orf228"    "CALM1"       "CCL4"        "CD55"        "CD81"       
 [56] "CD8B"        "CFL1"        "CMC1"        "DNAJB6"      "EEF1D"      
 [61] "EEF2"        "EIF1"        "EIF3E"       "EIF3K"       "EIF4B"      
 [66] "FAIM3"       "FLNA"        "FOXP1"       "FTL"         "GLTSCR2"    
 [71] "ID2"         "KLRB1"       "NACA"        "NDFIP1"      "NEAT1"      
 [76] "NPM1"        "PFDN5"       "PPA1"        "PPP2R5C"     "PTPN7"      
 [81] "RHOH"        "RIC3"        "RPL17"       "RPL24"       "RPL39"      
 [86] "RPL4"        "RPS10"       "RPS11"       "RPS24"       "RPS4X"      
 [91] "S100A6"      "SH3YL1"      "SLA"         "SMDT1"       "SRSF2"      
 [96] "SRSF7"       "SYNE1"       "TESPA1"      "TOMM7"       "TUBA4A"     
[101] "TXN"         "UBA52"       "UQCRB"       "ZEB2"        "ATP5A1"     
[106] "CD48"        "EIF3L"       "FAU"         "GPX1"        "IPO7"       
[111] "QARS"        "RNASE6"      "RPL7A"       "RPS9"        "S100A11"    
[116] "TKT"         "TNFAIP2"     "ACTB"        "ARPC2"       "B2M"        
[121] "EEF1B2"      "GNB2L1"      "KLRC1"       "RPL15"       "RPL18"      
[126] "RPL23"       "RPL29"       "RPL30"       "RPL34"       "RPL35"      
[131] "RPL37"       "RPL37A"      "RPL38"       "RPL6"        "RPLP0"      
[136] "RPS16"       "RPS20"       "RPS27A"      "RPS3"        "RPS5"       
[141] "RPSA"

In [136]:
### Retrieve gene positions an map to the data

In [137]:
mart = useEnsembl(biomart = "genes", dataset = "hsapiens_gene_ensembl")

In [138]:
geneSet =  unique(coloc_examples_rheomatoid_arthritis$cogene)

In [139]:
resultTable = biomaRt::getBM(attributes = c("start_position","end_position","description", 'hgnc_symbol', 'chromosome_name'),       
                  filters    = "hgnc_symbol",       
                  values     = geneSet,         
                  mart       = mart)     

In [140]:
nrow(resultTable)

[1] 170

In [141]:
length(unique(resultTable$hgnc_symbol))

[1] 132

In [142]:
### Filter out duplicate mappings

In [143]:
filter = resultTable %>% group_by(hgnc_symbol) %>% count() %>% filter(n >= 2)

In [144]:
filter = filter$hgnc_symbol

In [145]:
resultTable = resultTable[!resultTable$hgnc_symbol %in% filter,]

In [146]:
unique(resultTable$chromosome_name)

[1] "3"  "10" "7"  "5"  "15" "22" "2"  "12" "1"  "14" "11" "20" "16" "19" "17"
[16] "8"  "X"  "21" "4"  "18" "9"  "6"

In [147]:
head(resultTable,2)

,start_position,end_position,description,hgnc_symbol,chromosome_name
,<int>,<int>,<chr>,<chr>,<chr>
1,51968510,51983409,abhydrolase domain containing 14B [Source:HGNC Symbol;Acc:HGNC:28235],ABHD14B,3
2,114431112,114768061,actin binding LIM protein 1 [Source:HGNC Symbol;Acc:HGNC:78],ABLIM1,10


In [148]:
head(coloc_examples_rheomatoid_arthritis,2)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter,cogene
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
4259,Rheumatoid Arthritis,B_RPS26___RPL11__RPS26,B_RPS26,0.9035670,B,RPL11_RPS26,878,PP.H4.abf,RPL11
4269,Rheumatoid Arthritis,B_RPS26___RPL35A__RPS26,B_RPS26,0.9008136,B,RPL35A_RPS26,878,PP.H4.abf,RPL35A


In [149]:
coloc_examples_rheomatoid_arthritis = merge(coloc_examples_rheomatoid_arthritis, resultTable, by.x = 'cogene', by.y = 'hgnc_symbol', all.x = TRUE)

In [150]:
head(coloc_examples_rheomatoid_arthritis[is.na(coloc_examples_rheomatoid_arthritis$chromosome_name),])

,cogene,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter,start_position,end_position,description,chromosome_name
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<chr>
5,AIF1,Rheumatoid Arthritis,CD4T_RPS26___AIF1__RPS26,CD4T_RPS26,0.9269214,CD4T,AIF1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
6,AIF1,Rheumatoid Arthritis,CD8T_RPS26___AIF1__RPS26,CD8T_RPS26,0.9590870,CD8T,AIF1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
13,ATP5A1,Rheumatoid Arthritis,monocyte_RPS26___ATP5A1__RPS26,monocyte_RPS26,0.9595247,monocyte,ATP5A1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
14,B2M,Rheumatoid Arthritis,NK_RPS26___B2M__RPS26,NK_RPS26,0.9481042,NK,B2M_RPS26,879,PP.H4.abf,NA,NA,NA,NA
19,C1orf228,Rheumatoid Arthritis,CD8T_RPS26___C1orf228__RPS26,CD8T_RPS26,0.9100051,CD8T,C1orf228_RPS26,881,PP.H4.abf,NA,NA,NA,NA
21,CCL4,Rheumatoid Arthritis,CD8T_RPS26___CCL4__RPS26,CD8T_RPS26,0.9169790,CD8T,CCL4_RPS26,882,PP.H4.abf,NA,NA,NA,NA


In [151]:
#head(coloc_examples_rheomatoid_arthritis,4)

In [152]:
nrow(coloc_examples_rheomatoid_arthritis)

[1] 152

In [153]:
write.table(coloc_examples_rheomatoid_arthritis, file = paste0(path, "/colocalization_results/", "CD4T_RPS26_Rheomatoid_Arthritis_Colocalization_CoeGenes.csv"), append =FALSE, sep = ",", row.names = FALSE, col.names =TRUE)